https://www.researchgate.net/publication/220345072_Semantic_Text_Similarity_Using_Corpus-Based_Word_Similarity_and_String_Similarity
#threshold is 0.6 best for do this task

# Test GPT Model

In [3]:
!pip install python-crfsuite

  Using cached python_crfsuite-0.9.11-cp312-cp312-win_amd64.whl.metadata (4.4 kB)
Using cached python_crfsuite-0.9.11-cp312-cp312-win_amd64.whl (301 kB)


In [5]:
from pythainlp import sent_tokenize

text = "เดี๋ยววันนี้คุณหมอจะมีการจ่ายยาทั้งหมด2รายการนะครับมียาลอราทาดีน400มิลลิกรัมเป็นยาสำหรับบรรเทาอาการแพ้รับประทานครั้งละ1เม็ดวันละ1ครั้งหลังอาหารทันทีตอนเช้าจำนวน30เม็ดครับ"
sentences = sent_tokenize(text, engine="crfcut")  # ใช้โมเดล CRF เพื่อแบ่งประโยค
print(sentences)
# Output: ['ข้อความยาวๆที่ต้องการแบ่งเป็นประโยค', 'เช่นมีการวัดคะแนนแบบchecklistเป็นประโยคและคะแนนที่ได้อยู่']

['เดี๋ยววันนี้คุณหมอจะมีการจ่ายยาทั้งหมด2รายการนะครับมียาลอราทาดีน400มิลลิกรัมเป็นยาสำหรับบรรเทาอาการแพ้รับประทานครั้งละ1เม็ดวันละ1ครั้งหลังอาหารทันทีตอนเช้าจำนวน30เม็ดครับ']


In [29]:
import psycopg2
from psycopg2.extras import RealDictCursor

def get_db_connection():
    conn = psycopg2.connect(
        dbname='ScoringT',
        user='postgres',
        password='123456789',
        host='localhost'
    )
    return conn

selection_situ = 24

def get_data():
    conn = get_db_connection()
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute('SELECT check_sent,score,prerequired,topic FROM "having" h,situation s,checklist cl where h."ID_situ" = s."ID_situ" and cl."ID_checklist" = h."ID_checklist" and s."ID_situ"=%s order by cl.topic,cl."ID_checklist"',(selection_situ,))
    processing = cursor.fetchall()
    cursor.close()
    conn.close()
    return processing

def get_name_patient():
    conn = get_db_connection()
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute('SELECT fname_patient,lname_patient FROM situation s where s."ID_situ"=%s',(selection_situ,))
    processing = cursor.fetchall()
    cursor.close()
    conn.close()
    return processing

data = get_data()
checklist = [item['check_sent'] for item in data]
topic = [item['topic'] for item in data]
score = [item['score'] for item in data]
request = [item['prerequired'] for item in data]

name_patient = get_name_patient()
fname_patient = [item['fname_patient'] for item in name_patient]
lname_patient = [item['lname_patient'] for item in name_patient]
temp = checklist[2].split('fname')
temp.insert(1,fname_patient[0])
checklist[2] = ''.join(temp)
temp = checklist[3].split('fullname')
temp.insert(1,fname_patient[0])
temp.insert(2,lname_patient[0])
checklist[3] = ''.join(temp)

std_name = "สมชายยิ่งใหญ่"
checklist[0] = checklist[0]+std_name

In [34]:
from transformers import pipeline

def check_medicine_existence(medicine_keyword, context):
    # สร้างคำถามให้ชัดเจนพร้อมรูปแบบคำตอบที่ต้องการ
    question = f"ข้อความนี้กล่าวถึง{medicine_keyword}หรือไม่? ให้ตอบว่าใช่หรือไม่ใช่"
    
    # โหลดโมเดลครั้งเดียวเพื่อประสิทธิภาพ (ควรโหลดนอกฟังก์ชันถ้าใช้ซ้ำๆ)
    if not hasattr(check_medicine_existence, "model"):
        check_medicine_existence.model = pipeline(
            "question-answering", 
            model='wicharnkeisei/thai-xlm-roberta-base-squad2'
        )
    
    # ดำเนินการหาคำตอบ
    result = check_medicine_existence.model(
        question=question,
        context=context,
        max_answer_len=15,  # จำกัดความยาวคำตอบเพื่อความแม่นยำ
        handle_impossible_answer=True  # เปิดใช้งานการตรวจคำตอบที่เป็นไปไม่ได้
    )
    
    # ตรวจสอบความน่าเชื่อถือและประมวลผลคำตอบ
    if result['score'] > 0.5:  # เพิ่ม threshold ความเชื่อมั่น
        # ตรวจสอบทั้งคำตอบและคำถามย้อนกลับ
        is_positive = any(word in result['answer'] for word in ["ใช่", "มี", "กล่าวถึง"])
        return "ใช่" if is_positive else "ไม่ใช่", result['score']
    else:
        return "ไม่ใช่", result['score']
    
# ตัวอย่างการใช้งาน
context = """ยานี้ชื่อพาราเซตามอน้ำเชื่อม120มิลลิกรัมต่อ5มิลลิลิตร1ขวดใช้เพื่อบรรเทาอาการไข้และปวดรับประทานครั้งละ2มิลลิลิตรครึ่งทุก6ชั่วโมงเวลามีอาการไข้หรือปวดยานี้ไม่ควรรับประทานต่อเนื่องเกิน5วัน"""

for medicine_to_check in checklist:
    answer, confidence = check_medicine_existence(medicine_to_check, context)
    print(f"{medicine_to_check}: {answer} (ความมั่นใจ: {confidence:.2%})")

Device set to use cuda:0


แนะนำตัวเภสัชกรสมชายยิ่งใหญ่: ไม่ใช่ (ความมั่นใจ: 96.22%)
ผู้ป่วยชื่อนามสกุลอะไร: ไม่ใช่ (ความมั่นใจ: 87.05%)
ผู้ป่วยชื่อสวยนามสกุลอะไร: ไม่ใช่ (ความมั่นใจ: 92.58%)
ผู้ป่วยชื่อสวยสราญลมใช่ไหม: ไม่ใช่ (ความมั่นใจ: 87.12%)
ผู้ป่วยมีประวัติแพ้ยาอะไรไหม: ไม่ใช่ (ความมั่นใจ: 73.31%)
ผู้ป่วยแพ้ยามีอาการผื่นคันใช่ไหม: ไม่ใช่ (ความมั่นใจ: 92.05%)
พาราเซตามอล: ไม่ใช่ (ความมั่นใจ: 60.81%)
120มิลลิกรัมใน5มิลลิลิตร: ไม่ใช่ (ความมั่นใจ: 42.83%)
ใช้เพื่อบรรเทาอาการไข้และปวด: ไม่ใช่ (ความมั่นใจ: 2.51%)
รับประทานครั้งละ2จุด5มิลลิลิตร: ไม่ใช่ (ความมั่นใจ: 29.06%)
ทุก6ชั่วโมง: ไม่ใช่ (ความมั่นใจ: 6.88%)
เมื่อเวลามีอาการไข้หรือปวด: ไม่ใช่ (ความมั่นใจ: 9.96%)
ยานี้ไม่ควรรับประทานต่อเนื่องเกิน5วัน: ใช่ (ความมั่นใจ: 64.79%)
อะมอกซีซิลิน: ไม่ใช่ (ความมั่นใจ: 84.01%)
ความแรง500มิลลิกรัม: ไม่ใช่ (ความมั่นใจ: 94.60%)
จำนวน30เม็ด: ไม่ใช่ (ความมั่นใจ: 86.54%)
ใช้รักษาอาการติดเชื้อที่หู: ไม่ใช่ (ความมั่นใจ: 99.50%)
รับประทานครั้งละ1เม็ด: ไม่ใช่ (ความมั่นใจ: 45.20%)
วันละ3ครั้ง: ไม่ใช่ (ความมั่นใจ: 88.67%)
หลังอาหา

In [20]:
from transformers import pipeline

qa_model = pipeline("question-answering", "timpal0l/mdeberta-v3-base-squad2")
question = "มี การบอก ชื่อยา พาราเซตามอล ไหม"
context = "วันนี้มารับยา3ตัวนะคะยานี้ ชื่อ พาราเซตามอล น้ำเชื่อม 120มิลลิกรัมต่อ5มิลลิลิตร1ขวดใช้เพื่อบรรเทาอาการไข้และปวดรับประทานครั้งละ2มิลลิลิตรครึ่งทุก6ชั่วโมงเวลามีอาการไข้หรือปวดยานี้ไม่ควรรับประทานต่อเนื่องเกิน5วันส่วนยานี้ชื่ออะมอกซีซิลินความแรง500มิลลิกรัมจำนวน30เม็ดใช้รักษาอาการติดเชื้อที่หูรับประทานครั้งละ1เม็ดวันละ3ครั้งหลังอาหารเช้ากลางวันเย็นยานี้ควรรับประทานติดต่อกันทุกวันจนหมดและยานี้ชื่อเด็กซ์ออฟจำนวน1ขวดใช้สำหรับหยอดหูเพื่อรักษาอาการติดเชื้อที่หูหยอด3หยดที่หูข้างซ้ายวันละ3ครั้งหลังอาหารเช้ากลางวันเย็นซึ่งยานี้มีวิธีการใช้ในลักษณะพิเศษเดี๋ยวฉันสอนวิธีการใช้งานให้นะคะ"
qa_model(question = question, context = context)

Device set to use cuda:0


{'score': 0.16739019751548767,
 'start': 53,
 'end': 580,
 'answer': ' 120มิลลิกรัมต่อ5มิลลิลิตร1ขวดใช้เพื่อบรรเทาอาการไข้และปวดรับประทานครั้งละ2มิลลิลิตรครึ่งทุก6ชั่วโมงเวลามีอาการไข้หรือปวดยานี้ไม่ควรรับประทานต่อเนื่องเกิน5วันส่วนยานี้ชื่ออะมอกซีซิลินความแรง500มิลลิกรัมจำนวน30เม็ดใช้รักษาอาการติดเชื้อที่หูรับประทานครั้งละ1เม็ดวันละ3ครั้งหลังอาหารเช้ากลางวันเย็นยานี้ควรรับประทานติดต่อกันทุกวันจนหมดและยานี้ชื่อเด็กซ์ออฟจำนวน1ขวดใช้สำหรับหยอดหูเพื่อรักษาอาการติดเชื้อที่หูหยอด3หยดที่หูข้างซ้ายวันละ3ครั้งหลังอาหารเช้ากลางวันเย็นซึ่งยานี้มีวิธีการใช้ในลักษณะพิเศษเดี๋ยวฉันสอนวิธีการใช้งานให้นะคะ'}

In [4]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Number of GPUs available
print(torch.cuda.get_device_name(0))  # Name of the first GPU

True
1
NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [1]:
!nvidia-smi


Fri Mar  7 16:31:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.26                 Driver Version: 566.26         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   42C    P0              7W /   95W |       0MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
torch.backends.cuda.enable_flash_sdp(False)
torch.backends.cuda.enable_math_sdp(True)
torch.backends.cuda.enable_mem_efficient_sdp(False)

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load model and tokenizer
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Data for grading
student_response = "นักศึกษาเทน้ำเกลือใส่แก้วแล้วใช้กระบอกฉีดยาดูดน้ำเกลือ จากนั้นให้ผู้ป่วยก้มหน้าและเอียงหัวไปทางขวา"
grading_criteria = "นั่งหรือยืนก้มหน้าเล็กน้อย เอนศีรษะไปด้านใดด้านหนึ่ง"

# Construct prompt
prompt = f"""ข้อความ: "{student_response}"\n
คำถาม: "{grading_criteria}"\n
ข้อความนี้เกี่ยวข้องกับคำถามหรือไม่? กรุณาตอบ "มี" หรือ "ไม่มี" และอธิบายเหตุผลสั้นๆ:
"""

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate response with controlled length
output = model.generate(**inputs, max_length=200)

# Decode response
response_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(response_text)

OutOfMemoryError: CUDA out of memory. Tried to allocate 892.00 MiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 12.38 GiB is allocated by PyTorch, and 215.76 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# test word in sentence similarity

In [1]:
from sentence_transformers import SentenceTransformer, util

# โหลดโมเดลสำหรับสร้าง Text Embedding
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

s = ['ความแรง400มิลลิกรัม', "400มิลลิกรัม",]

# สร้าง Embedding
sent_embedding = model.encode(s)

# คำนวณ Cosine Similarity
cos_sim = util.pytorch_cos_sim(sent_embedding, sent_embedding)

# แสดงผลลัพธ์
print(cos_sim)

c:\Users\LENOVO\Desktop\DemoP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\LENOVO\Desktop\DemoP\.venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LENOVO\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an a

tensor([[1.0000, 0.8402],
        [0.8402, 1.0000]])


# don't work solution

In [100]:
import psycopg2
from psycopg2.extras import RealDictCursor

def get_db_connection():
    conn = psycopg2.connect(
        dbname='ScoringT',
        user='postgres',
        password='123456789',
        host='localhost'
    )
    return conn

In [101]:
def get_data():
    conn = get_db_connection()
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute('SELECT check_sent,score,prerequired,topic FROM "having" h,situation s,checklist cl where h."ID_situ" = s."ID_situ" and cl."ID_checklist" = h."ID_checklist" and s."ID_situ"=1 order by cl.topic')
    processing = cursor.fetchall()
    cursor.close()
    conn.close()
    return processing

In [102]:
data = get_data()
checklist = [item['check_sent'] for item in data]
topic = [item['topic'] for item in data]
score = [item['score'] for item in data]
request = [item['prerequired'] for item in data]

In [103]:
text = [
    "สวัสดีครับผมเภสัชกรสมชายยิ่งใหญ่คนไข้ชื่อนามสกุลอะไรครับ",
    "คนไข้มาด้วยอาการคันจมูกใช่ไหมครับ",
    "คนไข้มีโรคประจำตัวอะไรไหมครับ",
    "คนไข้มีประวัติการแพ้ยาอะไรไหมครับ",
    "มีอาการแพ้ยาแบบผื่นขึ้นแล้วคันใช่ไหมครับ",
    "เดี๋ยววันนี้คุณหมอจะมีการจ่ายยาทั้งหมด2รายการนะครับมียาลอราทาดีน400มิลลิกรัมเป็นยาสำหรับบรรเทาอาการแพ้รับประทานครั้งละ1เม็ดวันละ1ครั้งหลังอาหารทันทีตอนเช้าจำนวน30เม็ดครับ",
    "ส่วนอีกยาเป็นนอร์มอลซาไลน์นะครับขนาด1000มิลลิลิตรจำนวน2ขวดเป็นยาสำหรับล้างจมูกนะครับให้ล้างจมูกทั้ง2ข้างวันละ2ครั้งเช้าเย็น",
    "เดี๋ยวเภสัชขอแนะนำวิธีการใช้ยาสำหรับล้างจมูกนะครับหาแก้วสำหรับเทน้ำเกลือ0.9เปอร์เซ็นต์แล้วใช้กระบอกฉีดยาขนาด20ถึง50มิลลิลิตรดูดน้ำเกลือจนเต็มหลอดจากนั้นก้มหน้าลงเล็กน้อยเอียงหัวไปด้านหนึ่งกลั้นหายใจเพื่อไม่ให้สำลักฉีกน้ำเกลือประมาณ10-20ซีซีเข้ารูจมูกให้น้ำเกลือไหลออกทางจมูกอีกข้างใช้ลูกยางแดงดูดน้ำมูกในจมูกออกหรือสั่งน้ำมูกออกทำซ้ำหลายหลายครั้งในแต่ละข้างจนน้ำเกลือไหลออกทางจมูกจนไม่มีน้ำมูกหลังใช้งานเสร็จให้ล้างกระบวกฉีกยาให้สะอาดและทิ้งไว้ให้แห้งสนิท",
    "อาจใช้จุกล้างจมูกสวมที่ปลายกระบอกฉีดยาทำให้ล้างจมูกง่ายขึ้นถ้าแสบจมูกในช่วงแรกเป็นภาวะปกติไม่มีอันตรายตอนล้างจมูกอาจมีน้ำเกลือไหลลงคอทำให้รู้รึกเค็มหลังล้างจมูกเสร็จคนไข้สามารถกลั้วคอด้วยน้ำเปล่าได้เพื่อลดความเค็มลงส่วนกระบอกฉีกยาไม่ควรใช้ร่วมกับผู้อื่น",
]

In [104]:
text = text[0]
text

'สวัสดีครับผมเภสัชกรสมชายยิ่งใหญ่คนไข้ชื่อนามสกุลอะไรครับ'

In [8]:
from pythainlp.tokenize import word_tokenize

# Tokenize checklist และเก็บใน dictionary (เก็บ key เป็น string)
cl_token = {item: word_tokenize(item, keep_whitespace=False) for item in checklist}

# Tokenize input text
input_tokens = word_tokenize(text, keep_whitespace=False)

# สร้าง dictionary สำหรับเก็บผลลัพธ์
matched_dict = {key: [] for key in cl_token}

used_words = set()
for word in input_tokens:
    for key, sublist in cl_token.items():
        if word in sublist and word not in used_words:
            matched_dict[key].append(word)
            used_words.add(word)  # ป้องกันคำซ้ำ

# ลบคำซ้ำในแต่ละหัวข้อของ matched_dict โดยรักษาลำดับ
for key in matched_dict:
    matched_dict[key] = []
    used_words_in_key = set()  # ชุดสำหรับคำที่พบแล้วในแต่ละหัวข้อ
    for word in input_tokens:
        for cl_word in cl_token[key]:
            if word in cl_word and word not in used_words_in_key:
                matched_dict[key].append(word)
                used_words_in_key.add(word)

# ลบหัวข้อที่ไม่มีคำตรงกัน
matched_dict = {key: words for key, words in matched_dict.items() if words}

split_sent = []
# แสดงผล
for key, words in matched_dict.items():
    print(f"📌 {key}: {''.join(words)}")
    split_sent.append(''.join(words))


📌 แนะนำตัวเภสัชกร: เภสัชกร
📌 ผู้ป่วยชื่อนามสกุลอะไร: ชื่อนามสกุลอะไร
📌 ผู้ป่วยชื่อพิมนามสกุลอะไร: ชื่อนามสกุลอะไร
📌 ผู้ป่วยชื่อพิมมั่นใจใช่ไหม: ชื่อ
📌 ผู้ป่วยมีประวัติแพ้ยาอะไรไหม: อะไร


In [9]:
print(split_sent)

['เภสัชกร', 'ชื่อนามสกุลอะไร', 'ชื่อนามสกุลอะไร', 'ชื่อ', 'อะไร']


In [10]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
threshold = 0.6
recording = []
count = 0
matched_topics = {-1} # Track matched topics to avoid duplicate scoring

# Track whether each checklist item is matched
checklist_status = [False] * len(checklist)

# Encode the checklist sentences
checklit_embeddings = model.encode(checklist)

# Iterate through split_sent and check for similarity
for sent in split_sent:
    sent_embedding = model.encode(sent)
    similarities = util.cos_sim(sent_embedding, checklit_embeddings)

    for i, check in enumerate(checklist):
        if similarities[0][i] > threshold:
            best_match_index = i
            matched_checklist_item = checklist[best_match_index]
            matched_topic = topic[best_match_index]
            matched_score = score[best_match_index]
            required_topic = request[best_match_index]

            # Ensure dependencies are met
            if required_topic != -1 and required_topic not in matched_topics:
                continue  # Skip if required topic was not mentioned first

            checklist_status[best_match_index] = True  # Mark as matched

            # Add score only if the topic has not been counted before
            if matched_topic not in matched_topics:
                count += matched_score
                matched_topics.add(matched_topic)

            recording.append({
                "checklist_item": matched_checklist_item,
                "matched_sentence": sent,
                "similarity_score": similarities[0][i],
                "topic": matched_topic,
                "score": matched_score
            })

# Save results to an Excel file
df = pd.DataFrame(recording)
df.to_excel("recording_results.xlsx", index=False)

# Print the recorded matches
print(recording)
print("Checklist Status:")
for i,check in enumerate(checklist):
    print(check,":",checklist_status[i])
print("Total Score:", count)

[{'checklist_item': 'แนะนำตัวเภสัชกร', 'matched_sentence': 'เภสัชกร', 'similarity_score': tensor(0.9346), 'topic': 0, 'score': 0.5}, {'checklist_item': 'ผู้ป่วยชื่อนามสกุลอะไร', 'matched_sentence': 'เภสัชกร', 'similarity_score': tensor(0.6141), 'topic': 1, 'score': 1.0}, {'checklist_item': 'ผู้ป่วยชื่อนามสกุลอะไร', 'matched_sentence': 'ชื่อนามสกุลอะไร', 'similarity_score': tensor(0.6362), 'topic': 1, 'score': 1.0}, {'checklist_item': 'ผู้ป่วยชื่อพิมนามสกุลอะไร', 'matched_sentence': 'ชื่อนามสกุลอะไร', 'similarity_score': tensor(0.6471), 'topic': 1, 'score': 0.5}, {'checklist_item': 'ผู้ป่วยชื่อนามสกุลอะไร', 'matched_sentence': 'ชื่อนามสกุลอะไร', 'similarity_score': tensor(0.6362), 'topic': 1, 'score': 1.0}, {'checklist_item': 'ผู้ป่วยชื่อพิมนามสกุลอะไร', 'matched_sentence': 'ชื่อนามสกุลอะไร', 'similarity_score': tensor(0.6471), 'topic': 1, 'score': 0.5}]
Checklist Status:
แนะนำตัวเภสัชกร : True
ผู้ป่วยชื่อนามสกุลอะไร : True
ผู้ป่วยชื่อพิมนามสกุลอะไร : True
ผู้ป่วยชื่อพิมมั่นใจใช่ไหม : F

In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

model = SentenceTransformer("trained_model")
threshold = 0.6
recording = []
count = 0
matched_topics = {-1} # Track matched topics to avoid duplicate scoring

# Track whether each checklist item is matched
checklist_status = [False] * len(checklist)

# Encode the checklist sentences
checklit_embeddings = model.encode(checklist)

# Iterate through split_sent and check for similarity
for sent in split_sent:
    sent_embedding = model.encode(sent)
    similarities = util.cos_sim(sent_embedding, checklit_embeddings)

    for i, check in enumerate(checklist):
        if similarities[0][i] > threshold:
            best_match_index = i
            matched_checklist_item = checklist[best_match_index]
            matched_topic = topic[best_match_index]
            matched_score = score[best_match_index]
            required_topic = request[best_match_index]

            # Ensure dependencies are met
            if required_topic != -1 and required_topic not in matched_topics:
                continue  # Skip if required topic was not mentioned first

            checklist_status[best_match_index] = True  # Mark as matched

            # Add score only if the topic has not been counted before
            if matched_topic not in matched_topics:
                count += matched_score
                matched_topics.add(matched_topic)

            recording.append({
                "checklist_item": matched_checklist_item,
                "matched_sentence": sent,
                "similarity_score": similarities[0][i],
                "topic": matched_topic,
                "score": matched_score
            })

# Save results to an Excel file
df = pd.DataFrame(recording)
df.to_excel("recording_results.xlsx", index=False)

# Print the recorded matches
print(recording)
print("Checklist Status:")
for i,check in enumerate(checklist):
    print(check,":",checklist_status[i])
print("Total Score:", count)

[{'checklist_item': 'เป็นเภสัชกร', 'matched_sentence': 'เภสัช', 'similarity_score': tensor(0.8488), 'topic': 0, 'score': 0.5}, {'checklist_item': 'ใช้จุกล้างจมูกสวมที่ปลายกระบอกฉีดยาช่วยให้ล้างจมูกได้ง่ายขึ้น', 'matched_sentence': 'ใช้สำหรับล้างจมูก', 'similarity_score': tensor(0.7831), 'topic': 25, 'score': 0.5}, {'checklist_item': 'เทน้ำเกลือ0.9เปอร์เซ็นต์ใส่ถ้วยหรือแก้วที่เตรียมไว้แล้วใช้กระบอกฉีดยาขนาด20ถึง50มิลลิลิตรดูดน้ำเกลือจนเต็ม', 'matched_sentence': 'ใช้ยาแก้วเทน้ำเกลือ0.9เปอร์เซ็นต์แล้วกระบอกฉีดยาขนาด20ถึง50มิลลิลิตรดูดจนเต็มหรือกระไว้', 'similarity_score': tensor(0.9464), 'topic': 18, 'score': 0.5}, {'checklist_item': 'นั่งหรือยืนก้มหน้าเล็กน้อยเอนศีรษะไปด้านใดด้านหนึ่ง', 'matched_sentence': 'ก้มหน้าเล็กน้อยไปด้านหนึ่งหรือ', 'similarity_score': tensor(0.8079), 'topic': 19, 'score': 0.5}, {'checklist_item': 'ค่อยค่อยฉีดน้ำเกลือประมาณ10-20ซีซีเข้าไปในรูจมูกด้านบนให้น้ำเกลือไหลออกทางรูจมูกอีกข้าง', 'matched_sentence': 'จมูกน้ำเกลือ20ไปด้านให้ประมาณ10-ซีเข้ารูไหลออกทางอีกข้างใ

# new solution _before word sentence in checklist for tokenize sentence

In [1]:
import psycopg2
from psycopg2.extras import RealDictCursor

def get_db_connection():
    conn = psycopg2.connect(
        dbname='ScoringT',
        user='postgres',
        password='123456789',
        host='localhost'
    )
    return conn

In [163]:
selection_situ = 24

In [164]:
def get_data():
    conn = get_db_connection()
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute('SELECT check_sent,score,prerequired,topic FROM "having" h,situation s,checklist cl where h."ID_situ" = s."ID_situ" and cl."ID_checklist" = h."ID_checklist" and s."ID_situ"=%s order by cl.topic,cl."ID_checklist"',(selection_situ,))
    processing = cursor.fetchall()
    cursor.close()
    conn.close()
    return processing

In [165]:
data = get_data()
checklist = [item['check_sent'] for item in data]
topic = [item['topic'] for item in data]
score = [item['score'] for item in data]
request = [item['prerequired'] for item in data]

In [166]:
total_score = 0
pass_topics = set()

for i in range(len(data)):
    current_topic = data[i]['topic']
    current_score = data[i]['score']
    
    if current_topic not in pass_topics:
        total_score += current_score
        pass_topics.add(current_topic)

print("Total score (ไม่ซ้ำ topic):", total_score)

Total score (ไม่ซ้ำ topic): 10.0


In [167]:
def get_name_patient():
    conn = get_db_connection()
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute('SELECT fname_patient,lname_patient FROM situation s where s."ID_situ"=%s',(selection_situ,))
    processing = cursor.fetchall()
    cursor.close()
    conn.close()
    return processing

In [168]:
name_patient = get_name_patient()
fname_patient = [item['fname_patient'] for item in name_patient]
lname_patient = [item['lname_patient'] for item in name_patient]
temp = checklist[2].split('fname')
temp.insert(1,fname_patient[0])
checklist[2] = ''.join(temp)
temp = checklist[3].split('fullname')
temp.insert(1,fname_patient[0])
temp.insert(2,lname_patient[0])
checklist[3] = ''.join(temp)

In [169]:
std_name = "สมชายยิ่งใหญ่"
checklist[0] = checklist[0]+std_name
checklist

['แนะนำตัวเภสัชกรสมชายยิ่งใหญ่',
 'ผู้ป่วยชื่อนามสกุลอะไร',
 'ผู้ป่วยชื่อสวยนามสกุลอะไร',
 'ผู้ป่วยชื่อสวยสราญลมใช่ไหม',
 'ผู้ป่วยมีประวัติแพ้ยาอะไรไหม',
 'ผู้ป่วยแพ้ยามีอาการผื่นคันใช่ไหม',
 'พาราเซตามอล',
 '120มิลลิกรัมใน5มิลลิลิตร',
 'ใช้เพื่อบรรเทาอาการไข้และปวด',
 'รับประทานครั้งละ2จุด5มิลลิลิตร',
 'ทุก6ชั่วโมง',
 'เมื่อเวลามีอาการไข้หรือปวด',
 'ยานี้ไม่ควรรับประทานต่อเนื่องเกิน5วัน',
 'อะมอกซีซิลิน',
 'ความแรง500มิลลิกรัม',
 'จำนวน30เม็ด',
 'ใช้รักษาอาการติดเชื้อที่หู',
 'รับประทานครั้งละ1เม็ด',
 'วันละ3ครั้ง',
 'หลังอาหารเช้ากลางวันเย็น',
 'ควรรับประทานติดต่อกันทุกวันจนหมด',
 'เด็กซ์ออฟ',
 'ใช้รักษาหูชั้นกลางอักเสบ',
 'หยอดหูซ้าย3หยด',
 'วันละ3ครั้ง',
 'หลังอาหารเช้ากลางวันเย็น',
 'ล้างมือให้สะอาด',
 'นอนตะแคงหรือนั่งเอียงศีรษะให้หูข้างที่จะหยอดยาอยู่ด้านบน',
 'ให้ดึงใบหูข้างที่จะหยอดยาไปด้านหลังและดึงขึ้นด้านบน',
 'หยอดยาเข้าไปในหูตามที่แพทย์สั่ง',
 'ระวังอย่าเอาหลอดหยดสอดเข้าไปในรูหู',
 'เอียงตะแคงอยู่ท่าเดิมประมาณ3ถึง5นาที',
 'อาจเอาสำลีใส่ในรูหูไว้อีกระยะหนึ่งเพื่อไม่ให้ยาไห

In [16]:
text = [
    "วันนี้ มา รับ ยา 3 ตัว นะ ครับ อัน นี้ เป็น ยา ไอ บู โปร เพ นค วาม แรง 400 มิลลิกรัม จำนวน 10 เม็ด ใช้ เพื่อ บรรเทา อาการ ปวด รับประทาน ครั้ง ละ 1 เม็ด วัน ละ 3 ครั้งหลัง อาหารเช้า กลางวัน เย็น ยา นี้ ควร รับประทาน หลัง อาหาร ทันที ส่วน ยา นี้ ชื่อ อะ มอ ก ซี ซิ ลิ นค วาม แรง 500 มิลลิกรัม จำนวน 30 เม็ด ใช้ รักษา อาการ ติดเชื้อ ที่ หู รับประทาน ครั้ง ละ 1 เม็ด วัน ละ 3 ครั้งหลัง อาหารเช้า กลางวัน เย็น ยา นี้ ควร รับประทาน ติดต่อกัน ทุกวัน จน หมด",
    "จากนั้น ให้ ดึง ใบ หู ที่จะ หยอด ยา ไป ไว้ ด้านหลัง และ ดึง ขึ้น ด้านบน โดย หยอด ยา เข้าไป ใน หู ตามที่ แพทย์ สั่ง ระวัง อย่า เอา หยอด สอด เข้าไป ใน รูหู ให้ คนไข้ เอียง ตะแคง อยู่ ท่า เดิม ประมาณ 3 ถึง 5 นาที ซึ่ง อาจจะ เอา สำนิท ใส่ ไว้ ใน รูหู อีก ระยะ หนึ่ง เพื่อ ไม่ ให้ ยา ไหล ออกมา ได้",
    "ส่วนอีกยาเป็นน้ำเกลือ1000มิลลิลิตรจำนวน2ขวดเป็นยาสำหรับล้างจมูกนะครับให้ล้างจมูกทั้ง2ข้างวันละ2ครั้งเช้าเย็น",
    "เดี๋ยวเภสัชขอแนะนำวิธีการใช้ยาสำหรับล้างจมูกนะครับหาแก้วสำหรับเทน้ำเกลือแล้วใช้กระบอกฉีดยาขนาด20ถึง50มิลลิลิตรดูดน้ำเกลือจนเต็มหลอดจากนั้นก้มหน้าลงเล็กน้อยเอียงหัวไปด้านหนึ่งกลั้นหายใจเพื่อไม่ให้สำลักฉีกน้ำเกลือประมาณ10ถึง20ซีซีเข้ารูจมูกให้น้ำเกลือไหลออกทางจมูกอีกข้างใช้ลูกยางแดงดูดน้ำมูกในจมูกออกหรือสั่งน้ำมูกออกทำซ้ำหลายหลายครั้งในแต่ละข้างจนน้ำเกลือไหลออกทางจมูกจนไม่มีน้ำมูกหลังใช้งานเสร็จให้ล้างกระบวกฉีกยาให้สะอาดและทิ้งไว้ให้แห้งสนิท",
    "อาจใช้จุกล้างจมูกสวมที่ปลายกระบอกฉีดยาทำให้ล้างจมูกง่ายขึ้นถ้าแสบจมูกในช่วงแรกเป็นภาวะปกติไม่มีอันตรายตอนล้างจมูกอาจมีน้ำเกลือไหลลงคอทำให้รู้รึกเค็มหลังล้างจมูกเสร็จคนไข้สามารถกลั้วคอด้วยน้ำเปล่าได้เพื่อลดความเค็มลงส่วนกระบอกฉีกยาไม่ควรใช้ร่วมกับผู้อื่น",
    "วันนี้หมอจัดยาให้ทั้งหมด3รายการนะคะขออธิบายการใช้ให้ฟังก่อนค่ะรายการแรกเป็นยาพาราเซตามอลชนิดน้ำเชื่อม1ขวดมีความแรงของยา120มิลลิกรัมต่อ5มิลลิลิตรใช้ลดไข้และบรรเทาอาการไม่สบายให้รับประทานครั้งละ2.5มิลลิลิตรทุก6ชั่วโมงหรือเมื่อมีไข้ยานี้ไม่ควรใช้ต่อเนื่องเกิน5วันค่ะรายการที่2คืออะมอกซีซิลลินขนาด500มิลลิกรัมจำนวน30เม็ดเป็นยาปฏิชีวนะสำหรับรักษาการติดเชื้อที่หูให้ทานครั้งละ1เม็ดวันละ3เวลาหลังอาหารเช้ากลางวันและเย็นยานี้ต้องทานให้ครบตามจำนวนที่แพทย์สั่งถึงแม้อาการจะดีขึ้นแล้วก็ตามนะคะรายการสุดท้ายคือยาหยอดหูชื่อเด็กซ์ออฟ1ขวดใช้หยอดที่หูข้างซ้ายครั้งละ3หยดวันละ3ครั้งหลังอาหารเช้ากลางวันและเย็นยานี้มีวิธีการใช้เฉพาะเดี๋ยวหนูจะสอนให้นะคะ",
    "ก่อนใช้ยาควรล้างมือให้สะอาดใช้ไม้พันสำลีเช็ดทำความสะอาดหนองหรือสิ่งสกปรกบริเวณรูหูทั้งก่อนและหลังการหยอดยาจากนั้นหยิบขวดยาขึ้นมาเขย่าขวดก่อนใช้ทุกครั้งให้นั่งเอียงศีรษะหูข้างที่ต้องการหยอดควรอยู่ด้านบนใช้นิ้วดึงใบหูไปทางด้านหลังและยกขึ้นเล็กน้อยหยอดยาลงในหูตามจำนวนที่กำหนดโดยระวังอย่าให้ปลายหลอดหยดยาเข้าไปในรูหูหลังจากหยอดยาแล้วให้นอนตะแคงหรือเอียงศีรษะค้างไว้ในท่าเดิมประมาณ3–5นาทีอาจใช้สำลีปิดรูหูไว้ชั่วคราวเพื่อป้องกันไม่ให้ยาหยดไหลย้อนออกมา",
    "หากคุณเก็บยานี้ไว้ในตู้เย็นก่อนใช้ควรกำขวดยาไว้ในอุ้งมือสักครู่เพื่อปรับอุณหภูมิของยาให้ใกล้เคียงกับร่างกายก่อนหยอดค่ะ",
    "ขั้นตอนการฉีดยา:ทำความสะอาดบริเวณที่จะฉีดด้วยสำลีแอลกอฮอล์ใช้มือข้าง1ดึงผิวหนังให้ตึงส่วนอีกข้างถือกระบอกฉีดยาให้คล้ายปากกาแล้วแทงเข็มเข้าไปที่ผิวหนังอย่างรวดเร็วโดยเอียงเข็มประมาณ45ถึง90องศาค่อยกดฉีดยาช้าจนสุดแล้วถอนเข็มออกอย่างรวดเร็วจากนั้นใช้สำลีชุบแอลกอฮอล์กดที่ตำแหน่งฉีดยาเบาโดยไม่ต้องคลึงหลังใช้แล้วให้ทิ้งกระบอกฉีดยาและเข็มอย่างมิดชิดในภาชนะที่ปลอดภัย",
    "ตำแหน่งที่แนะนำในการฉีดยาได้แก่หน้าท้องต้นแขนต้นขาและบริเวณสะโพกครับพยายามเปลี่ยนจุดฉีดบ้างอย่าฉีดจุดเดิมซ้ำโดยให้ห่างจากตำแหน่งเดิมประมาณ1นิ้วส่วนการเก็บรักษาอินซูลิน:ขวดยาที่ยังไม่เปิดใช้ควรเก็บไว้ในตู้เย็นถ้าเปิดใช้แล้วให้ใช้ให้หมดภายใน30วันและเก็บในอุณหภูมิห้องที่ไม่โดนแสงหรือความร้อนโดยตรงหลังฉีดยาควรสังเกตอาการเช่นมือสั่นใจสั่นเหงื่อออกมากหรือรู้สึกหิวผิดปกติหากเกิดอาการเหล่านี้อาจเป็นภาวะน้ำตาลในเลือดต่ำให้รีบรับประทานอาหารหรือของหวานและปรึกษาแพทย์หรือเภสัชกรทันทีนะครับ",
]

In [33]:
sentence = text[4]
sentence

'อาจใช้จุกล้างจมูกสวมที่ปลายกระบอกฉีดยาทำให้ล้างจมูกง่ายขึ้นถ้าแสบจมูกในช่วงแรกเป็นภาวะปกติไม่มีอันตรายตอนล้างจมูกอาจมีน้ำเกลือไหลลงคอทำให้รู้รึกเค็มหลังล้างจมูกเสร็จคนไข้สามารถกลั้วคอด้วยน้ำเปล่าได้เพื่อลดความเค็มลงส่วนกระบอกฉีกยาไม่ควรใช้ร่วมกับผู้อื่น'

## start idea word tokenize

In [64]:
# custom_words = [
#     "คนไข้",
#     "พาราเซตามอล","120มิลลิกรัมใน5มิลลิลิตร","จำนวน1ขวด","ใช้เพื่อบรรเทาอาการไข้และปวด","รับประทานครั้งละ2จุด5มิลลิลิตร","ทุก6ชั่วโมง","เวลามีอาการไข้หรือปวด","ไม่ควรรับประทานต่อเนื่องเกิน5วัน",
#     "อะมอกซีซิลิน","ความแรง500มิลลิกรัม","จำนวน30เม็ด","รักษาอาการติดเชื้อที่หู","ควรรับประทานติดต่อกันทุกวันจนหมด",
#     "เด็กซ์ออฟ","จำนวน1ขวด","หยอด3หยดที่หูข้างซ้าย","วันละ3ครั้ง",
#     "ล้างมือให้สะอาด","นอนตะแคงหรือนั่งเอียงศีรษะให้หูข้างที่จะหยอดยาอยู่ด้านบน","ให้ดึงใบหูข้างที่จะหยอดยาไปด้านหลังและดึงขึ้นด้านบน",
#     "หยอดยาเข้าไปในหูตามที่แพทย์สั่ง","ระวังอย่าเอาหลอดหยดสอดเข้าไปในรูหู","เอียงตะแคงอยู่ท่าเดิมประมาณ3ถึง5นาที","สำลีใส่ในรูหูไว้อีกระยะหนึ่งเพื่อไม่ให้ยาไหลออกมา",
#     "ให้เขย่าขวดก่อนใช้ทุกครั้ง","ใช้ไม้พันสำลีเช็ดหนองในใบหูหรือรูหูให้สะอาดก่อนและหลังหยอดยา","หากเก็บไว้ในตู้เย็นก่อนใช้ยาให้กำหลอดยาไว้ในอุ้งมือสักครู่",
#     "เพื่อปรับอุณหภูมิให้ใกล้เคียงกับร่างกายก่อนหยอด","ซึ่งยานี้มีวิธีการใช้ในลักษณะพิเศษเดี๋ยวฉันสอนวิธีการใช้งานให้นะคะ","เขย่า",
# ]

In [65]:
# from pythainlp.util import Trie

# new_words = [
#     "ผู้ป่วย","คนไข้",
#     "พาราเซตามอล","120มิลลิกรัมต่อ5มิลลิลิตร","120มิลลิกรัมใน5มิลลิลิตร","1ขวด","จำนวน1ขวด","บรรเทาอาการไข้","บรรเทาอาการไข้และปวด","ใช้เพื่อบรรเทาอาการไข้และปวด","รับประทานครั้งละ2มิลลิลิตรครึ่ง","รับประทานครั้งละ2จุด5มิลลิลิตร","ทุก6ชั่วโมง","เวลามีอาการไข้หรือปวด","ไม่ควรรับประทานต่อเนื่องเกิน5วัน",
#     "อะมอกซีซิลิน","500มิลลิกรัม","ความแรง500มิลลิกรัม","30เม็ด","จำนวน30เม็ด","รักษาอาการติดเชื้อที่หู","ควรรับประทานติดต่อกันทุกวันจนหมด","ควรทานจนหมด",
#     "เด็กซ์ออฟ","1ขวด","จำนวน1ขวด","หยอด3หยดที่หูข้างซ้าย","วันละ3ครั้ง",
#     "ล้างมือให้สะอาด","นอนตะแคงหรือนั่งเอียงศีรษะให้หูข้างที่จะหยอดยาอยู่ด้านบน","ให้ดึงใบหูข้างที่จะหยอดยาไปด้านหลังและดึงขึ้นด้านบน",
#     "หยอดยาเข้าไปในหูตามที่แพทย์สั่ง","ระวังอย่าเอาหลอดหยดสอดเข้าไปในรูหู","เอียงตะแคงอยู่ท่าเดิมประมาณ3ถึง5นาที","สำลีใส่ในรูหูไว้อีกระยะหนึ่งเพื่อไม่ให้ยาไหลออกมา",
#     "ให้เขย่าขวดก่อนใช้ทุกครั้ง","ใช้ไม้พันสำลีเช็ดหนองในใบหูหรือรูหูให้สะอาดก่อนและหลังหยอดยา","หากเก็บไว้ในตู้เย็นก่อนใช้ยาให้กำหลอดยาไว้ในอุ้งมือสักครู่",
#     "เพื่อปรับอุณหภูมิให้ใกล้เคียงกับร่างกายก่อนหยอด","ซึ่งยานี้มีวิธีการใช้ในลักษณะพิเศษเดี๋ยวฉันสอนวิธีการใช้งานให้นะคะ","เขย่า",
# ]

# trie = Trie(new_words)

In [20]:
def get_custom_words():
    conn = get_db_connection()
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute('SELECT sem_word_token FROM public.semantic_token_situ sts where sts."ID_situ"=%s',(selection_situ,))
    processing = cursor.fetchall()
    processing = [row['sem_word_token'] for row in processing]
    cursor.close()
    conn.close()
    return processing

In [21]:
def get_new_words():
    conn = get_db_connection()
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute('SELECT word_token FROM public.token_situ ts where ts."ID_situ"=%s',(selection_situ,))
    processing = cursor.fetchall()
    processing = [row['word_token'] for row in processing]
    cursor.close()
    conn.close()
    return processing

In [36]:
custom_words = get_custom_words()
new_words = get_new_words()
from pythainlp.util import Trie
trie = Trie(new_words)

In [37]:
import torch
from pythainlp.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util

torch.cuda.empty_cache()
torch.cuda.ipc_collect()

# โหลดโมเดล
model = SentenceTransformer("token_model")

words_to_delete = {
    "อืม", "อ่า", "เอ่อ", "อือ", "เอ้อ", "แบบ", "ไง", "มั้ง","เออ",
    "งั้น", "ก็", "อ่ะ", "เหรอ", "นะ", "น่ะ", "ล่ะ", "แหละ", "หรอก",
    "หรือยัง", "ใช่มั้ย", "ไหม", "มั้ย", "แฮะ", "อุ๊ย", "ว้าว", "อ้าว", "เฮ้ย",
    "อ๊ะ", "อิอิ", "555", "เอะ", "เอิ่ม", "โอ้", "โอ๊ะ", "หืม", "โธ่", "จ๊ะ", "จ้ะ", "เจ๊า","นะ","เนี่ย",
    }  # คำที่ต้องการลบ

words_to_replace = {}

sentence = sentence.replace('\n', '')
sentence = sentence.replace('สอง', '2')
sentence = sentence.replace('หนึ่ง', '1')
sentence = sentence.replace('ๆ', '')
sentence = sentence.replace('-', '')
sentence = sentence.replace('.','จุด')
sentence = sentence.replace(' ', '')   # ลบช่องว่างทั้งหมด

# ตัดคำ
token_basic_words = word_tokenize(sentence, keep_whitespace=False)

# ลบคำที่อยู่ใน words_to_delete
filtered_words = [word for word in token_basic_words if word not in words_to_delete]

# รวมคำกลับเป็นประโยค
new_sentence = ''.join(filtered_words)

# ตัดคำประโยค
words = word_tokenize(new_sentence,keep_whitespace=False,custom_dict=trie)

# คำนวณความคล้ายคลึงของแต่ละคำในประโยคกับ custom_words
sentence_embeddings = model.encode(words)
custom_embeddings = model.encode(custom_words)

# ตรวจหาคำที่มี similarity สูงกับ custom_words
split_positions = []
for i, word_embedding in enumerate(sentence_embeddings):
    similarity = util.cos_sim(word_embedding, custom_embeddings).argmax().item()
    similarities = util.cos_sim(word_embedding, custom_embeddings)  # คำนวณ similarity matrix
    max_index = similarities.argmax().item()
    
    if similarities[0][max_index].item() > 0.6:  # เช็ค threshold
        print(f"{words[i]} มีความคล้ายกับหนึ่งใน {custom_words[max_index]} (similarity: {similarities[0][max_index].item():.2f})")
        split_positions.append(i)

print(words)
print(split_positions)

# แบ่งประโยคตามตำแหน่งที่พบ
split_sentences = []
prev_idx = 0
for idx in split_positions:
    if idx != 0:
        split_sentences.append("".join(words[prev_idx:idx]))
        prev_idx = idx
split_sentences.append("".join(words[prev_idx:]))

# แสดงผลลัพธ์
print(split_sentences)

อาจ มีความคล้ายกับหนึ่งใน อาจ (similarity: 1.00)
ใช้ มีความคล้ายกับหนึ่งใน ใช้ (similarity: 1.00)
กระบอกฉีดยา มีความคล้ายกับหนึ่งใน กระบอกฉีดยา (similarity: 1.00)
ถ้า มีความคล้ายกับหนึ่งใน ถ้า (similarity: 1.00)
อาจ มีความคล้ายกับหนึ่งใน อาจ (similarity: 1.00)
มี มีความคล้ายกับหนึ่งใน หาก (similarity: 0.73)
น้ำเกลือ มีความคล้ายกับหนึ่งใน น้ำเกลือ (similarity: 1.00)
หลัง มีความคล้ายกับหนึ่งใน หลัง (similarity: 1.00)
ล้างจมูกเสร็จ มีความคล้ายกับหนึ่งใน เสร็จ (similarity: 0.72)
คนไข้ มีความคล้ายกับหนึ่งใน คนไข้ (similarity: 1.00)
กระบอกฉีกยาไม่ควรใช้ร่วมกับผู้อื่น มีความคล้ายกับหนึ่งใน กระบอกฉีกยาไม่ควรใช้ร่วมกับผู้อื่นครับ (similarity: 1.00)
['อาจ', 'ใช้', 'จุกล้างจมูกสวมที่ปลาย', 'กระบอกฉีดยา', 'ทำให้ล้างจมูกง่ายขึ้น', 'ถ้า', 'แสบจมูกในช่วงแรกเป็นภาวะปกติไม่มีอันตรายตอนล้างจมูก', 'อาจ', 'มี', 'น้ำเกลือ', 'ไหลลงคอทำให้รู้รึกเค็ม', 'หลัง', 'ล้างจมูกเสร็จ', 'คนไข้', 'สามารถกลั้วคอด้วยน้ำเปล่าได้เพื่อลดความเค็มลง', 'ส่วน', 'กระบอกฉีกยาไม่ควรใช้ร่วมกับผู้อื่น']
[0, 1, 3, 5, 7, 8, 9, 11, 12, 

In [58]:
# import torch
# from sentence_transformers import SentenceTransformer, util
# import pandas as pd

# torch.cuda.empty_cache()

# model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
# threshold = 0.6
# recording = []
# count = 0
# matched_topics = {-1} # Track matched topics to avoid duplicate scoring

# # Track whether each checklist item is matched
# checklist_status = [False] * len(checklist)

# # Encode the checklist sentences
# checklit_embeddings = model.encode(checklist)

# # Iterate through split_sent and check for similarity
# for sent in split_sentences:
#     sent_embedding = model.encode(sent)
#     similarities = util.cos_sim(sent_embedding, checklit_embeddings)

#     # best_match_index = similarities[0].argmax().item()  # Get index of highest similarity
#     # best_score = similarities[0][best_match_index].item()

#     for i, check in enumerate(checklist):
#         if similarities[0][i] > threshold:
#             best_match_index = i
#             matched_checklist_item = checklist[best_match_index]
#             matched_topic = topic[best_match_index]
#             matched_score = score[best_match_index]
#             required_topic = request[best_match_index]

#             # Ensure dependencies are met
#             if required_topic != -1 and required_topic not in matched_topics:
#                 continue  # Skip if required topic was not mentioned first

#             checklist_status[best_match_index] = True  # Mark as matched

#             # Add score only if the topic has not been counted before
#             if matched_topic not in matched_topics:
#                 count += matched_score
#                 matched_topics.add(matched_topic)

#             recording.append({
#                 "checklist_item": matched_checklist_item,
#                 "matched_sentence": sent,
#                 "similarity_score": similarities[0][i],
#                 "topic": matched_topic,
#                 "score": matched_score
#             })

# # Save results to an Excel file
# df = pd.DataFrame(recording)
# df.to_excel("recording_results.xlsx", index=False)

# # Print the recorded matches
# print(recording)
# print("Checklist Status:")
# for i,check in enumerate(checklist):
#     print(check,":",checklist_status[i])
# print("Total Score:", count)

In [38]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from evaluate import load
from jiwer import wer
cer = load("cer")

torch.cuda.empty_cache()
torch.cuda.ipc_collect()

model = SentenceTransformer('trained_model')
threshold = 0.6
recording = []
count = 0
matched_topics = {-1} # Track matched topics to avoid duplicate scoring

# Track whether each checklist item is matched
checklist_status = [False] * len(checklist)

# Encode the checklist sentences
checklit_embeddings = model.encode(checklist)

# Iterate through split_sent and check for similarity
for sent in split_sentences:
    sent_embedding = model.encode(sent)
    similarities = util.cos_sim(sent_embedding, checklit_embeddings)

    # best_match_index = similarities[0].argmax().item()  # Get index of highest similarity
    # best_score = similarities[0][best_match_index].item()

    for i, check in enumerate(checklist):
        if similarities[0][i] > threshold:
            best_match_index = i
            matched_checklist_item = checklist[best_match_index]
            matched_topic = topic[best_match_index]
            matched_score = score[best_match_index]
            required_topic = request[best_match_index]
            
            cer_score = cer.compute(predictions=[sent], references=[matched_checklist_item])
            tokenized_sent = ' '.join(word_tokenize(sent, keep_whitespace=False))
            tokenized_ref = ' '.join(word_tokenize(matched_checklist_item, keep_whitespace=False))
            wer_score = wer(tokenized_sent, tokenized_ref)

            # Ensure dependencies are met
            if required_topic != -1 and required_topic not in matched_topics:
                continue  # Skip if required topic was not mentioned first

            checklist_status[best_match_index] = True  # Mark as matched

            # Add score only if the topic has not been counted before
            if matched_topic not in matched_topics:
                count += matched_score
                matched_topics.add(matched_topic)

            recording.append({
                "checklist_item": matched_checklist_item,
                "matched_sentence": sent,
                "similarity_score": similarities[0][i],
                "CER_score": cer_score,
                "WER_score": wer_score,
                "topic": matched_topic,
                "score": matched_score
            })

# Save results to an Excel file
df = pd.DataFrame(recording)
df.to_excel("recording_results.xlsx", index=False)

# Print the recorded matches
print(recording)
print("Checklist Status:")
for i,check in enumerate(checklist):
    print(check,":",checklist_status[i])
print("Total Score:", count)

[{'checklist_item': 'ใช้จุกล้างจมูกสวมที่ปลายกระบอกฉีดยา', 'matched_sentence': 'ใช้จุกล้างจมูกสวมที่ปลาย', 'similarity_score': tensor(0.8982), 'CER_score': 0.3142857142857143, 'WER_score': 0.2857142857142857, 'topic': 25, 'score': 0.25}, {'checklist_item': 'ช่วยให้ล้างจมูกได้ง่ายขึ้น', 'matched_sentence': 'กระบอกฉีดยาทำให้ล้างจมูกง่ายขึ้น', 'similarity_score': tensor(0.6022), 'CER_score': 0.5769230769230769, 'WER_score': 0.5714285714285714, 'topic': 30, 'score': 0.25}, {'checklist_item': 'อาจมีอาการระคายเคืองหรือแสบจมูกในระยะแรกเป็นภาวะปกติไม่มีอันตราย', 'matched_sentence': 'ถ้าแสบจมูกในช่วงแรกเป็นภาวะปกติไม่มีอันตรายตอนล้างจมูก', 'similarity_score': tensor(0.8084), 'CER_score': 0.59375, 'WER_score': 0.6428571428571429, 'topic': 26, 'score': 0.5}, {'checklist_item': 'น้ำเกลือล้างจมูกส่วนหนึ่งอาจไหลลงคอทำให้รู้สึกเค็มในคอขณะล้างจมูกหรือภายหลังล้างจมูกเสร็จ', 'matched_sentence': 'น้ำเกลือไหลลงคอทำให้รู้รึกเค็ม', 'similarity_score': tensor(0.7545), 'CER_score': 0.6741573033707865, 'WER_sc

In [19]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

torch.cuda.empty_cache()
torch.cuda.ipc_collect()

model = SentenceTransformer('trained_model')
threshold = 0.6
recording = []
count = 0
matched_topics = {-1} # Track matched topics to avoid duplicate scoring

# Track whether each checklist item is matched
checklist_status = [False] * len(checklist)

# Encode the checklist sentences
checklit_embeddings = model.encode(checklist)

# Iterate through split_sent and check for similarity
for doublecheck in range(0,2):
    for sent in split_sentences:
        sent_embedding = model.encode(sent)
        similarities = util.cos_sim(sent_embedding, checklit_embeddings)

        # best_match_index = similarities[0].argmax().item()  # Get index of highest similarity
        # best_score = similarities[0][best_match_index].item()

        for i, check in enumerate(checklist):
            if similarities[0][i] > threshold:
                best_match_index = i
                matched_checklist_item = checklist[best_match_index]
                matched_topic = topic[best_match_index]
                matched_score = score[best_match_index]
                required_topic = request[best_match_index]

                # Ensure dependencies are met
                if required_topic != -1 and required_topic not in matched_topics:
                    continue  # Skip if required topic was not mentioned first

                checklist_status[best_match_index] = True  # Mark as matched

                # Add score only if the topic has not been counted before
                if matched_topic not in matched_topics:
                    count += matched_score
                    matched_topics.add(matched_topic)

                recording.append({
                    "checklist_item": matched_checklist_item,
                    "matched_sentence": sent,
                    "similarity_score": similarities[0][i],
                    "topic": matched_topic,
                    "score": matched_score
                })

# Save results to an Excel file
df = pd.DataFrame(recording)
df.to_excel("recording_results.xlsx", index=False)

# Print the recorded matches
print(recording)
print("Checklist Status:")
for i,check in enumerate(checklist):
    print(check,":",checklist_status[i])
print("Total Score:", count)

[{'checklist_item': 'ให้ดึงใบหูข้างที่จะหยอดยาไปด้านหลังและดึงขึ้นด้านบน', 'matched_sentence': 'ให้ดึงใบหูที่จะหยอดยาไปไว้ด้านหลังและดึงขึ้นด้านบน', 'similarity_score': tensor(0.9903), 'topic': 27, 'score': 0.5}, {'checklist_item': 'หยอดยาเข้าไปในหูตามที่แพทย์สั่ง', 'matched_sentence': 'ให้ดึงใบหูที่จะหยอดยาไปไว้ด้านหลังและดึงขึ้นด้านบน', 'similarity_score': tensor(0.8179), 'topic': 28, 'score': 0.5}, {'checklist_item': 'ให้ดึงใบหูข้างที่จะหยอดยาไปด้านหลังและดึงขึ้นด้านบน', 'matched_sentence': 'หยอดยาเข้าไปในหูตามที่แพทย์สั่ง', 'similarity_score': tensor(0.8302), 'topic': 27, 'score': 0.5}, {'checklist_item': 'หยอดยาเข้าไปในหูตามที่แพทย์สั่ง', 'matched_sentence': 'หยอดยาเข้าไปในหูตามที่แพทย์สั่ง', 'similarity_score': tensor(1.), 'topic': 28, 'score': 0.5}, {'checklist_item': 'ระวังอย่าเอาหลอดหยดสอดเข้าไปในรูหู', 'matched_sentence': 'ระวังอย่าเอาหยอดสอดเข้าไปในรูหูให้', 'similarity_score': tensor(0.9682), 'topic': 29, 'score': 1.0}, {'checklist_item': 'เอียงตะแคงอยู่ท่าเดิมประมาณ3ถึง5นา

In [21]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

torch.cuda.empty_cache()
torch.cuda.ipc_collect()

model = SentenceTransformer('nomic_trained_model', trust_remote_code=True)
threshold = 0.6
recording = []
count = 0
matched_topics = {-1} # Track matched topics to avoid duplicate scoring

# Track whether each checklist item is matched
checklist_status = [False] * len(checklist)

# Encode the checklist sentences
checklit_embeddings = model.encode(checklist)

# Iterate through split_sent and check for similarity
for doublecheck in range(0,2):
    for sent in split_sentences:
        sent_embedding = model.encode(sent)
        similarities = util.cos_sim(sent_embedding, checklit_embeddings)

        # best_match_index = similarities[0].argmax().item()  # Get index of highest similarity
        # best_score = similarities[0][best_match_index].item()

        for i, check in enumerate(checklist):
            if similarities[0][i] > threshold:
                best_match_index = i
                matched_checklist_item = checklist[best_match_index]
                matched_topic = topic[best_match_index]
                matched_score = score[best_match_index]
                required_topic = request[best_match_index]

                # Ensure dependencies are met
                if required_topic != -1 and required_topic not in matched_topics:
                    continue  # Skip if required topic was not mentioned first

                checklist_status[best_match_index] = True  # Mark as matched

                # Add score only if the topic has not been counted before
                if matched_topic not in matched_topics:
                    count += matched_score
                    matched_topics.add(matched_topic)

                recording.append({
                    "checklist_item": matched_checklist_item,
                    "matched_sentence": sent,
                    "similarity_score": similarities[0][i],
                    "topic": matched_topic,
                    "score": matched_score
                })

# Save results to an Excel file
df = pd.DataFrame(recording)
df.to_excel("recording_results.xlsx", index=False)

# Print the recorded matches
print(recording)
print("Checklist Status:")
for i,check in enumerate(checklist):
    print(check,":",checklist_status[i])
print("Total Score:", count)

[{'checklist_item': 'เทน้ำเกลือ0.9เปอร์เซ็นต์ใส่ถ้วยหรือแก้วที่เตรียมไว้', 'matched_sentence': 'เทน้ำเกลือ0จุด9เปอร์เซ็นต์แล้ว', 'similarity_score': tensor(0.6209), 'topic': 18, 'score': 0.25}, {'checklist_item': 'ใช้กระบอกฉีดยาขนาด20ถึง50มิลลิลิตรดูดน้ำเกลือจนเต็ม', 'matched_sentence': 'กระบอกฉีดยาความจุประมาณ20ถึง50มิลลิลิตรดูดน้ำเกลือจนเต็มหลอด', 'similarity_score': tensor(0.8903), 'topic': 29, 'score': 0.25}, {'checklist_item': 'ใช้ลูกยางแดงดูดน้ำมูกในจมูกออกหรือสั่งน้ำมูกออก', 'matched_sentence': 'ใช้ลูกยางแดงดูดน้ำมูกให้หมดก่อนจากนั้นยืน', 'similarity_score': tensor(0.7456), 'topic': 22, 'score': 1.0}, {'checklist_item': 'นั่งหรือยืนก้มหน้าเล็กน้อยเอนศีรษะไปด้านใดด้านหนึ่ง', 'matched_sentence': 'ลงเล็กน้อยเอียงหัวไปด้านหนึ่ง', 'similarity_score': tensor(0.6478), 'topic': 19, 'score': 0.5}, {'checklist_item': 'กลั้นหายใจหรือหายใจทางปากเพื่อป้องกันการสำลัก', 'matched_sentence': 'กลั้นหายใจเพื่อไม่ให้สำลัก', 'similarity_score': tensor(0.8448), 'topic': 20, 'score': 1.0}, {'checklist

In [58]:
#token 792
#semantic token 636

# test code version 1

In [73]:
from pythainlp.tokenize import word_tokenize
from pythainlp.util import Trie

In [ ]:
# Define custom words
#custom_words = ["ครับ","ชื่อ","ยานี้ชื่อ","มิลลิกรัม", "เม็ด","ความแรง","รับประทานครั้งละ","ครั้ง","ขวด","จำนวน","วันละ","ควรรับประทานหลังอาหารทันที","ควรรับประทานติดต่อกันทุกวันจนหมด"]
# Create a Trie for custom dictionary
#trie = Trie(custom_words)

In [ ]:
#text = "วันนี้มารับยา3ตัวนะครับยานี้ชื่อไอบูโปรเพนความแรง400มิลลิกรัมจำนวน10เม็ดใช้เพื่อบรรเทาอาการปวดรับประทานครั้งละ1เม็ดวันละ3ครั้งหลังอาหารเช้ากลางวันเย็นยานี้ควรรับประทานหลังอาหารทันทีส่วนยานี้ชื่ออะมอกซีซิลินความแรง500มิลลิกรัมจำนวน30เม็ดใช้รักษาอาการติดเชื้อที่หูรับประทานครั้งละ1เม็ดวันละ3ครั้งหลังอาหารเช้ากลางวันเย็นยานี้ควรรับประทานติดต่อกันทุกวันจนหมดและยานี้ชื่อเด็กซ์ออฟจำนวน1ขวดใช้สำหรับหยอดหูเพื่อรักษาอาการติดเชื้อที่หูหยอด3หยดที่หูข้างซ้ายวันละ3ครั้งหลังอาหารเช้ากลางวันเย็น"
#text = "สวัสดีครับผมเป็นเภสัชกรชื่อสิทธิกรเฉลิมกิจที่ชัยคนไข้ชื่อนามสกุลอะไรครับคนไข้มีอาการปวดหูใช่ไหมครับ"
#text = "วันนี้มารับยา 3 ตัวนะครับยานี้ชื่อไอบูโพรเฟนความแรง 400 มิลลิกรัมจำนวน 10 เม็ดใช้เพื่อบรรเทาอาการปวดรับประทานครั้งละ 1 เม็ดวันละ 3 ครั้งหลังอาหารเช้ากลางวันเย                           ย็นยานี้ควรรับประทานหลังอาหารทันทีส่วนยานี้ชื่ออะม็อกซี่ซีเมนต์ความแรง 500 มิลลิกรัมจำนวน 30 เม็ดใช้รักษาอาการติดเชื้อที่หูรับประทานครั้งละ 1 เม็ดวันละ 3 ครั้ง                                   งหลังอาหารเช้ากลางวันเย็นยานี้ควรรับประทานติดต่อกันทุกวันจนหมดและยานี้ชื่อ Desktop จำนวน 1 ขวดใช้สำหรับหยอดหูเพื่อรักษาอาการติดเชื้อที่หูหยอด 3 หยดขี้หูข้างซ้า                                ายวันละ 3 ครั้งหลังอาหารเช้ากลางวันเย็น"

In [2]:
text = [
    "สวัสดีค่ะหนูเป็นเภสัชกรชื่อสมหญิงใจดีคนไข้ชื่อนามสกุลอะไรคะ",
    "คนไข้มาด้วยอาการปวดหูใช่ไหมคะ",
    """วันนี้มารับยา3ตัวนะค่ะยานี้ชื่อไอบูโปรเพนความแรง400มิลลิกรัมจำนวน10เม็ดใช้เพื่อบรรเทาอาการปวดรับประทานครั้งละ1เม็ดวันละ3ครั้งหลังอาหารเช้ากลางวันเย็นยานี้ควรรับประทานหลังอาหารทันทีส่วนยานี้ชื่ออะมอกซีซิลินความแรง500มิลลิกรัมจำนวน30เม็ดใช้รักษาอาการติดเชื้อที่หูรับประทานครั้งละ1เม็ดวันละ3ครั้งหลังอาหารเช้ากลางวันเย็นยานี้ควรรับประทานติดต่อกันทุกวันจนหมดและยานี้ชื่อเด็กซ์ออฟจำนวน1ขวดใช้สำหรับหยอดหูเพื่อรักษาอาการติดเชื้อที่หูหยอด3หยดที่หูข้างซ้ายวันละ3ครั้งหลังอาหารเช้ากลางวันเย็นซึ่งยานี้มีวิธีการใช้ในลักษณะพิเศษเดี๋ยวฉันสอนวิธีการใช้งานให้นะคะเริ่มด้วยการล้างมือให้สะอาดก่อนใช้ยาจากนั้นใช้ไม้พันสำลีเช็ดหนองในใบหูหรือรูหูให้สะอาดก่อนและหลังหยอดยาต่อมาให้หยิบยาจากกล่องมาถือไว้ที่มือและเขย่าขวดก่อนใช้ทุกครั้งถัดไปให้คนไข้นอนตะแคงหรือนั่งเอียงศีรษะให้หูข้างที่จะหยอดยาอยู่ด้านบนถัดมาให้ดึงใบหูข้างที่จะหยอดยาไปด้านหลังและดึงขึ้นด้านบนโดยหยอดยาเข้าไปในหูตามที่แพทย์สั่งระวังอย่าเอาหลอดหยดสอดเข้าไปในรูหูให้คนไข้เอียงตะแคงอยู่ท่าเดิมประมาณ3ถึง5นาทีซึ่งอาจจะเอาสำลีใส่ในรูหูไว้อีกระยะหนึ่งเพื่อไม่ให้ยาไหลออกมาหากเก็บยาหยอดหูไว้ในตู้เย็นก่อนใช้ยาให้กำหยอดยาไว้ในอุ้งมือสักครู่เพื่อปรับอุณหภูมิให้ใกล้เคียงกับร่างกายก่อนหยอด""",
]

In [3]:
# Define custom words
custom_words = [
    "คนไข้ชื่อนามสกุลอะไร","สวัสดี","ไอบูโปรเพน","ความแรง400มิลลิกรัม","จำนวน10เม็ด","รับประทานครั้งละ1เม็ด","วันละ3ครั้ง","หลังอาหารเช้ากลางวันเย็น",
    "ควรรับประทานหลังอาหารทันที","อะมอกซีซิลิน","ความแรง500มิลลิกรัม","จำนวน30เม็ด","ควรรับประทานติดต่อกันทุกวันจนหมด","เด็กซ์ออฟ","จำนวน1ขวด",
    "หยอด3หยดที่หูข้างซ้าย","วันละ3ครั้ง","ล้างมือให้สะอาด"
    ]
# Create a Trie for custom dictionary
trie = Trie(custom_words)

In [4]:
split_sent = word_tokenize(text[2], custom_dict=trie)

In [5]:
print(split_sent)

['วันนี้มารับยา', '3', 'ตัวนะค่ะยานี้ชื่อ', 'ไอบูโปรเพน', 'ความแรง400มิลลิกรัม', 'จำนวน10เม็ด', 'ใช้เพื่อบรรเทาอาการปวด', 'รับประทานครั้งละ1เม็ด', 'วันละ3ครั้ง', 'หลังอาหารเช้ากลางวันเย็น', 'ยานี้', 'ควรรับประทานหลังอาหารทันที', 'ส่วนยานี้ชื่อ', 'อะมอกซีซิลิน', 'ความแรง500มิลลิกรัม', 'จำนวน30เม็ด', 'ใช้รักษาอาการติดเชื้อที่หู', 'รับประทานครั้งละ1เม็ด', 'วันละ3ครั้ง', 'หลังอาหารเช้ากลางวันเย็น', 'ยานี้', 'ควรรับประทานติดต่อกันทุกวันจนหมด', 'และยานี้ชื่อ', 'เด็กซ์ออฟ', 'จำนวน1ขวด', 'ใช้สำหรับหยอดหูเพื่อรักษาอาการติดเชื้อที่หู', 'หยอด3หยดที่หูข้างซ้าย', 'วันละ3ครั้ง', 'หลังอาหารเช้ากลางวันเย็น', 'ซึ่งยานี้มีวิธีการใช้ในลักษณะพิเศษเดี๋ยวฉันสอนวิธีการใช้งานให้นะคะเริ่มด้วยการ', 'ล้างมือให้สะอาด', 'ก่อนใช้ยาจากนั้นใช้ไม้พันสำลีเช็ดหนองในใบหูหรือรูหูให้สะอาดก่อนและหลังหยอดยาต่อมาให้หยิบยาจากกล่องมาถือไว้ที่มือและเขย่าขวดก่อนใช้ทุกครั้งถัดไปให้คนไข้นอนตะแคงหรือนั่งเอียงศีรษะให้หูข้างที่จะหยอดยาอยู่ด้านบนถัดมาให้ดึงใบหูข้างที่จะหยอดยาไปด้านหลังและดึงขึ้นด้านบนโดยหยอดยาเข้าไปในหูตามที่แพทย์สั่งร

In [6]:
x = 3.5
MIN = 0
MAX = 5
normalize = (x - MIN)/(MAX - MIN)
print(normalize)

0.7


In [7]:
recording = []
threshold = 0.75

topic = [
    0,  # "ผมเป็นเป็นเภสัชกร"
    1,  # "ผู้ป่วยชื่อนามสกุลอะไร"
    1,  # "ผู้ป่วยชื่ออะไร"
    1,  # "ชาติ"
    1,  # "ชาติตระการตา"
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,  
    10,  
    11,  
    12, 
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
]

checklit = [
    "ผมเป็นเป็นเภสัชกร",
    "ผู้ป่วยชื่อนามสกุลอะไร",
    "ผู้ป่วยชื่ออะไร",
    "ผู้ป่วยชื่อชาตินามสกุลอะไร",
    "ผู้ป่วยชื่อชาติตระการตาใช่ไหม",
    "ไอบูโปรเพน",
    "ความแรง400มิลลิกรัม",
    "จำนวน10เม็ด",
    "ใช้เพื่อบรรเทาอาการปวด",
    "รับประทานครั้งละ1เม็ด",
    "วันละ3ครั้ง",
    "หลังอาหารเช้ากลางวันเย็น",
    "ควรรับประทานหลังอาหารทันที",
    "อะมอกซีซิลิน",
    "ความแรง500มิลลิกรัม",
    "จำนวน30เม็ด",
    "ใช้รักษาอาการติดเชื้อที่หู",
    "รับประทานครั้งละ1เม็ด",
    "วันละ3ครั้ง",
    "หลังอาหารเช้ากลางวันเย็น",
    "ควรรับประทานติดต่อกันทุกวันจนหมด",
    "เด็กซ์ออฟ",
    "ใช้รักษาหูชั้นกลางอักเสบ",
    "หยอดหูซ้าย3หยด",
    "วันละ3ครั้ง",
    "หลังอาหารเช้ากลางวันเย็น",
]

score = [
    1,    # "ผมเป็นเป็นเภสัชกร"
    1,    # "ผู้ป่วยชื่อนามสกุลอะไร"
    1,    # "ผู้ป่วยชื่ออะไร"
    0.5,  # "ชาติ"
    0.25, # "ชาติตระการตา"
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0, 
    0,
    0,
    0,
    0,
    0,   # เด็กซ์ออฟ
    0.25, # "ใช้รักษาหูชั้นกลางอักเสบ"
    0.25, # "หยอดหูซ้าย3หยด"
    0.25, # "วันละ3ครั้ง"
    0.25, # "เช้ากลางวันเย็น"
]

request = [
    -1,  # "ผมเป็นเป็นเภสัชกร"
    -1,  # "ผู้ป่วยชื่อนามสกุลอะไร"
    -1,  # "ผู้ป่วยชื่ออะไร"
    -1,  # "ชาติ"
    -1,  # "ชาติตระการตา"
    -1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    -1,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    -1,  # เด็กซ์ออฟ
    16,  # "ใช้รักษาหูชั้นกลางอักเสบ"
    17,  # "หยอดหูซ้าย3หยด"
    18,  # "วันละ3ครั้ง"
    19,  # "เช้ากลางวันเย็น"
]

In [8]:
print("topic = ",len(topic))
print("checklit = ",len(checklit))
print("request = ",len(request))

topic =  26
checklit =  26
request =  26


In [9]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

count = 0
matched_topics = {-1} # Track matched topics to avoid duplicate scoring

# Track whether each checklist item is matched
checklist_status = {check: False for check in checklit}

# Encode the checklist sentences
checklit_embeddings = model.encode(checklit)

# Iterate through split_sent and check for similarity
for sent in split_sent:
    sent_embedding = model.encode(sent)
    similarities = util.cos_sim(sent_embedding, checklit_embeddings)

    best_match_index = similarities[0].argmax().item()  # Get index of highest similarity
    best_score = similarities[0][best_match_index].item()

    if best_score > threshold:
        matched_checklist_item = checklit[best_match_index]
        matched_topic = topic[best_match_index]
        matched_score = score[best_match_index]
        required_topic = request[best_match_index]

       # Ensure dependencies are met
        if required_topic != -1 and required_topic not in matched_topics:
            continue  # Skip if required topic was not mentioned first

        checklist_status[matched_checklist_item] = True  # Mark as matched

        # Add score only if the topic has not been counted before
        if matched_topic not in matched_topics:
            count += matched_score
            matched_topics.add(matched_topic)

        recording.append({
            "checklist_item": matched_checklist_item,
            "matched_sentence": sent,
            "similarity_score": best_score,
            "topic": matched_topic,
            "score": matched_score
        })

# Print the recorded matches
print(recording)
print("Checklist Status:", checklist_status)
print("Total Score:", count)


[{'checklist_item': 'ไอบูโปรเพน', 'matched_sentence': 'ไอบูโปรเพน', 'similarity_score': 0.9999999403953552, 'topic': 2, 'score': 0}, {'checklist_item': 'ความแรง400มิลลิกรัม', 'matched_sentence': 'ความแรง400มิลลิกรัม', 'similarity_score': 1.0000001192092896, 'topic': 3, 'score': 0}, {'checklist_item': 'จำนวน10เม็ด', 'matched_sentence': 'จำนวน10เม็ด', 'similarity_score': 1.0, 'topic': 4, 'score': 0}, {'checklist_item': 'ใช้เพื่อบรรเทาอาการปวด', 'matched_sentence': 'ใช้เพื่อบรรเทาอาการปวด', 'similarity_score': 0.9999998807907104, 'topic': 5, 'score': 0}, {'checklist_item': 'รับประทานครั้งละ1เม็ด', 'matched_sentence': 'รับประทานครั้งละ1เม็ด', 'similarity_score': 0.9999999403953552, 'topic': 6, 'score': 0}, {'checklist_item': 'วันละ3ครั้ง', 'matched_sentence': 'วันละ3ครั้ง', 'similarity_score': 1.0, 'topic': 7, 'score': 0}, {'checklist_item': 'หลังอาหารเช้ากลางวันเย็น', 'matched_sentence': 'หลังอาหารเช้ากลางวันเย็น', 'similarity_score': 1.0, 'topic': 8, 'score': 0}, {'checklist_item': 'ควรร

In [93]:
from sentence_transformers import SentenceTransformer, util

# โหลดโมเดลสำหรับสร้าง Text Embedding
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
# กำหนดเกณฑ์ (เช่น 0.7)
threshold = 0.85

# ขั้นตอนมาตรฐาน
steps = [
    #check information
    "เป็นเภสัชกร",
    "นามสกุลอะไร",
    "ชาติ",
    "ชาติตระการตา",
    "ปวดหูข้างที่ข้างซ้าย",
    "ความดัน",
    "ตั้งครรภ์",
    #check drug
    "ไอบูโปรเพน",
    "ความแรง",
    "400",
    "จำนวน",
    "10",
    "ใช้เพื่อบรรเทาอาการปวด",
    "รับประทานครั้งละ",
    "1",
    "วันละ",
    "3",
    "หลังอาหารเช้ากลางวันเย็น",
    "ควรรับประทานหลังอาหารทันที",
    "อะมอกซีซิลิน",
    "ขนาดยา",
    "500",
    "จำนวน",
    "30",
    "ใช้รักษาอาการติดเชื้อที่หู",
    "รับประทานครั้งละ",
    "1",
    "วันละ",
    "3",
    "หลังอาหารเช้ากลางวันเย็น",
    "ควรรับประทานติดต่อกันทุกวันจนหมด",
    "เด็กซ์ออฟ",
    "จำนวน",
    "1",
    "ใช้สำหรับหยอดหูเพื่อรักษาอาการติดเชื้อที่หู",
    "3",
    "หูข้างซ้าย",
    "วันละ",
    "3",
    "หลังอาหารเช้ากลางวันเย็น",
    "มีวิธีการใช้ในลักษณะพิเศษดังนี้",
    #howtouse
    "ล้างมือให้สะอาด",
    "นอนตะแคงหรือนั่งเอียงศีรษะให้หูข้างที่จะหยอดยาอยู่ด้านบน",
    "ให้ดึงใบหูข้างที่จะหยอดยาไปด้านหลังและดึงขึ้นด้านบน",
    "หยอดยาเข้าไปในหูตามที่แพทย์สั่ง",
    "ระวังอย่าเอาหลอดหยดสอดเข้าไปในรูหู",
    "เอียงตะแคงอยู่ท่าเดิมประมาณ3ถึง5นาที",
    "เอาสำลีใส่ในรูหูไว้อีกระยะหนึ่งเพื่อไม่ให้ยาไหลออกมา",
    #addition
    "ให้เขย่าขวดก่อนใช้ทุกครั้ง",
    "ใช้ไม้พันสำลีเช็ดหนองในใบหูหรือรูหูให้สะอาดก่อนและหลังหยอดยา",
    "เก็บไว้ในตู้เย็นก่อนใช้ยาให้กำหลอดยาไว้ในอุ้งมือสักครู่",
    "เพื่อปรับอุณหภูมิให้ใกล้เคียงกับร่างกายก่อนหยอด",
]

# Track whether each step is covered by at least one sentence
coverage = {step: False for step in steps}
doublecheck = []

# แปลงขั้นตอนมาตรฐานและข้อความเภสัชกรเป็น Embedding
step_embeddings = model.encode(steps)

for sent in split_sent:
    sent_embedding = model.encode(sent)
    similarities = util.cos_sim(sent_embedding, step_embeddings)
    
    # อัปเดตผลลัพธ์หากเจอข้อความที่ตรงกับขั้นตอนมาตรฐาน
    for i, step in enumerate(steps):
        if similarities[0][i] > threshold:
            coverage[step] = True
            doublecheck.append(step)
            doublecheck.append(sent)
            doublecheck.append(similarities[0][i])

#แสดงผล
# for step, covered in coverage.items():
#     status = "ผ่าน" if covered else "ไม่ผ่าน"
#     print(f"{step}: {status}")

print(doublecheck)

[]


In [5]:
from sentence_transformers import SentenceTransformer, util

# โหลดโมเดลสำหรับสร้าง Text Embedding
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

s = ['ล้างจมูกทั้ง2ข้าง', "ล้างจมูกทั้งสองข้าง", "เช้าและเย็น","เช้าเย็น","ให้ดึงใบหูข้างที่จะหยอดยาไปด้านหลังและดึงขึ้นด้านบน","เอาหูด้านซ้ายขึ้นด้านบน"]

# สร้าง Embedding
sent_embedding = model.encode(s)

# คำนวณ Cosine Similarity
cos_sim = util.pytorch_cos_sim(sent_embedding, sent_embedding)

# แสดงผลลัพธ์
print(cos_sim)

tensor([[1.0000, 0.9775, 0.4064, 0.2396, 0.5751, 0.5104],
        [0.9775, 1.0000, 0.4477, 0.2841, 0.5666, 0.5100],
        [0.4064, 0.4477, 1.0000, 0.6624, 0.2733, 0.3100],
        [0.2396, 0.2841, 0.6624, 1.0000, 0.2770, 0.3379],
        [0.5751, 0.5666, 0.2733, 0.2770, 1.0000, 0.8012],
        [0.5104, 0.5100, 0.3100, 0.3379, 0.8012, 1.0000]])


In [10]:
from sentence_transformers import SentenceTransformer, util

# โหลดโมเดลสำหรับสร้าง Text Embedding
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

s = ['เทน้ำเกลือ0.9เปอร์เซ็นต์ใส่ถ้วยหรือแก้วที่เตรียมไว้แล้วใช้กระบอกฉีดยาขนาด20-50มิลลิลิตรดูดน้ำเกลือจนเต็ม','เทน้ำเกลือใส่ถ้วยหรือแก้วที่เตรียมไว้แล้วใช้กระบอกฉีดยาขนาด20-50มิลลิลิตรดูดน้ำเกลือจนเต็ม', "เทน้ำเกลือใส่ถ้วยที่เตรียมไว้แล้วใช้กระบอกฉีดยาดูดน้ำเกลือจนเต็มหยอด",
    "นั่งหรือยืนก้มหน้าเล็กน้อยเอนศีรษะไปด้านใดด้านหนึ่ง","ก้มหน้านิดหน่อยแล้วเอนศีรษะไปทางด้านใดด้านหนึ่ง"]

# สร้าง Embedding
sent_embedding = model.encode(s)

# คำนวณ Cosine Similarity
cos_sim = util.pytorch_cos_sim(sent_embedding, sent_embedding)

# แสดงผลลัพธ์
print(cos_sim)

tensor([[1.0000, 0.9401, 0.8331, 0.0737, 0.1158],
        [0.9401, 1.0000, 0.9091, 0.0889, 0.1377],
        [0.8331, 0.9091, 1.0000, 0.1590, 0.2501],
        [0.0737, 0.0889, 0.1590, 1.0000, 0.8800],
        [0.1158, 0.1377, 0.2501, 0.8800, 1.0000]])


# database version2

In [21]:
import psycopg2
from psycopg2.extras import RealDictCursor

def get_db_connection():
    conn = psycopg2.connect(
        dbname='Testing',
        user='postgres',
        password='123456789',
        host='localhost'
    )
    return conn

In [22]:
def get_data():
    conn = get_db_connection()
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute('SELECT * FROM processing order by topic')
    processing = cursor.fetchall()
    cursor.close()
    conn.close()
    return processing

In [23]:
data = get_data()
checklist = [item['checklist'] for item in data]
topic = [item['topic'] for item in data]
score = [item['score'] for item in data]
request = [item['prerequest'] for item in data]

In [24]:
from pythainlp.tokenize import word_tokenize
from pythainlp.util import Trie

In [25]:
text = [
    "สวัสดีค่ะหนูเป็นเภสัชกรชื่อสมหญิงใจดีคนไข้ชื่อนามสกุลอะไรคะ",
    "คนไข้มาด้วยอาการปวดหูใช่ไหมคะ",
    """วันนี้มารับยา3ตัวนะค่ะยานี้ชื่อไอบูโปรเพนความแรง400มิลลิกรัมจำนวน10เม็ดใช้เพื่อบรรเทาอาการปวดรับประทานครั้งละ1เม็ดวันละ3ครั้งหลังอาหารเช้ากลางวันเย็นยานี้ควรรับประทานหลังอาหารทันทีส่วนยานี้ชื่ออะมอกซีซิลินความแรง500มิลลิกรัมจำนวน30เม็ดใช้รักษาอาการติดเชื้อที่หูรับประทานครั้งละ1เม็ดวันละ3ครั้งหลังอาหารเช้ากลางวันเย็นยานี้ควรรับประทานติดต่อกันทุกวันจนหมดและยานี้ชื่อเด็กซ์ออฟจำนวน1ขวดใช้สำหรับหยอดหูเพื่อรักษาอาการติดเชื้อที่หูหยอด3หยดที่หูข้างซ้ายวันละ3ครั้งหลังอาหารเช้ากลางวันเย็นซึ่งยานี้มีวิธีการใช้ในลักษณะพิเศษเดี๋ยวฉันสอนวิธีการใช้งานให้นะคะเริ่มด้วยการล้างมือให้สะอาดก่อนใช้ยาจากนั้นใช้ไม้พันสำลีเช็ดหนองในใบหูหรือรูหูให้สะอาดก่อนและหลังหยอดยาต่อมาให้หยิบยาจากกล่องมาถือไว้ที่มือและเขย่าขวดก่อนใช้ทุกครั้งถัดไปให้คนไข้นอนตะแคงหรือนั่งเอียงศีรษะให้หูข้างที่จะหยอดยาอยู่ด้านบนถัดมาให้ดึงใบหูข้างที่จะหยอดยาไปด้านหลังและดึงขึ้นด้านบนโดยหยอดยาเข้าไปในหูตามที่แพทย์สั่งระวังอย่าเอาหลอดหยดสอดเข้าไปในรูหูให้คนไข้เอียงตะแคงอยู่ท่าเดิมประมาณ3ถึง5นาทีซึ่งอาจจะเอาสำลีใส่ในรูหูไว้อีกระยะหนึ่งเพื่อไม่ให้ยาไหลออกมาหากเก็บยาหยอดหูไว้ในตู้เย็นก่อนใช้ยาให้กำหยอดยาไว้ในอุ้งมือสักครู่เพื่อปรับอุณหภูมิให้ใกล้เคียงกับร่างกายก่อนหยอด""",
]

In [26]:
# Define custom words
custom_words = [
    "คนไข้ชื่อนามสกุลอะไร","สวัสดี","ไอบูโปรเพน","ความแรง400มิลลิกรัม","จำนวน10เม็ด","รับประทานครั้งละ1เม็ด","วันละ3ครั้ง","หลังอาหารเช้ากลางวันเย็น",
    "ควรรับประทานหลังอาหารทันที","อะมอกซีซิลิน","ความแรง500มิลลิกรัม","จำนวน30เม็ด","ควรรับประทานติดต่อกันทุกวันจนหมด","เด็กซ์ออฟ","จำนวน1ขวด",
    "หยอด3หยดที่หูข้างซ้าย","วันละ3ครั้ง","ล้างมือให้สะอาด","นอนตะแคงหรือนั่งเอียงศีรษะให้หูข้างที่จะหยอดยาอยู่ด้านบน","ให้ดึงใบหูข้างที่จะหยอดยาไปด้านหลังและดึงขึ้นด้านบน",
    "หยอดยาเข้าไปในหูตามที่แพทย์สั่ง","ระวังอย่าเอาหลอดหยดสอดเข้าไปในรูหู","เอียงตะแคงอยู่ท่าเดิมประมาณ3ถึง5นาที","สำลีใส่ในรูหูไว้อีกระยะหนึ่งเพื่อไม่ให้ยาไหลออกมา",
    "ให้เขย่าขวดก่อนใช้ทุกครั้ง","ใช้ไม้พันสำลีเช็ดหนองในใบหูหรือรูหูให้สะอาดก่อนและหลังหยอดยา","หากเก็บไว้ในตู้เย็นก่อนใช้ยาให้กำหลอดยาไว้ในอุ้งมือสักครู่",
    "เพื่อปรับอุณหภูมิให้ใกล้เคียงกับร่างกายก่อนหยอด",
    ]
# Create a Trie for custom dictionary
trie = Trie(custom_words)

In [27]:
split_sent = word_tokenize(text[2], custom_dict=trie)
print(split_sent)

['วันนี้มารับยา', '3', 'ตัวนะค่ะยานี้ชื่อ', 'ไอบูโปรเพน', 'ความแรง400มิลลิกรัม', 'จำนวน10เม็ด', 'ใช้เพื่อบรรเทาอาการปวด', 'รับประทานครั้งละ1เม็ด', 'วันละ3ครั้ง', 'หลังอาหารเช้ากลางวันเย็น', 'ยานี้', 'ควรรับประทานหลังอาหารทันที', 'ส่วนยานี้ชื่อ', 'อะมอกซีซิลิน', 'ความแรง500มิลลิกรัม', 'จำนวน30เม็ด', 'ใช้รักษาอาการติดเชื้อที่หู', 'รับประทานครั้งละ1เม็ด', 'วันละ3ครั้ง', 'หลังอาหารเช้ากลางวันเย็น', 'ยานี้', 'ควรรับประทานติดต่อกันทุกวันจนหมด', 'และยานี้ชื่อ', 'เด็กซ์ออฟ', 'จำนวน1ขวด', 'ใช้สำหรับหยอดหูเพื่อรักษาอาการติดเชื้อที่หู', 'หยอด3หยดที่หูข้างซ้าย', 'วันละ3ครั้ง', 'หลังอาหารเช้ากลางวันเย็น', 'ซึ่งยานี้มีวิธีการใช้ในลักษณะพิเศษเดี๋ยวฉันสอนวิธีการใช้งานให้นะคะเริ่มด้วยการ', 'ล้างมือให้สะอาด', 'ก่อนใช้ยาจากนั้น', 'ใช้ไม้พันสำลีเช็ดหนองในใบหูหรือรูหูให้สะอาดก่อนและหลังหยอดยา', 'ต่อมาให้หยิบยาจากกล่องมาถือไว้ที่มือและเขย่าขวดก่อนใช้ทุกครั้งถัดไปให้คนไข้', 'นอนตะแคงหรือนั่งเอียงศีรษะให้หูข้างที่จะหยอดยาอยู่ด้านบน', 'ถัดมา', 'ให้ดึงใบหูข้างที่จะหยอดยาไปด้านหลังและดึงขึ้นด้านบน', 'โดย', 'หยอด

test score

In [28]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
threshold = 0.75
recording = []
count = 0
matched_topics = {-1} # Track matched topics to avoid duplicate scoring

# Track whether each checklist item is matched
checklist_status = [False] * len(checklist)

# Encode the checklist sentences
checklit_embeddings = model.encode(checklist)

# Iterate through split_sent and check for similarity
for sent in split_sent:
    sent_embedding = model.encode(sent)
    similarities = util.cos_sim(sent_embedding, checklit_embeddings)

    # best_match_index = similarities[0].argmax().item()  # Get index of highest similarity
    # best_score = similarities[0][best_match_index].item()

    for i, check in enumerate(checklist):
        if similarities[0][i] > threshold:
            best_match_index = i
            matched_checklist_item = checklist[best_match_index]
            matched_topic = topic[best_match_index]
            matched_score = score[best_match_index]
            required_topic = request[best_match_index]

            # Ensure dependencies are met
            if required_topic != -1 and required_topic not in matched_topics:
                continue  # Skip if required topic was not mentioned first

            checklist_status[best_match_index] = True  # Mark as matched

            # Add score only if the topic has not been counted before
            if matched_topic not in matched_topics:
                count += matched_score
                matched_topics.add(matched_topic)

            recording.append({
                "checklist_item": matched_checklist_item,
                "matched_sentence": sent,
                "similarity_score": similarities[0][i],
                "topic": matched_topic,
                "score": matched_score
            })

# Save results to an Excel file
df = pd.DataFrame(recording)
df.to_excel("recording_results.xlsx", index=False)

# Print the recorded matches
print(recording)
print("Checklist Status:")
for i,check in enumerate(checklist):
    print(check,":",checklist_status[i])
print("Total Score:", count)

[{'checklist_item': 'ไอบูโปรเพน', 'matched_sentence': 'ไอบูโปรเพน', 'similarity_score': tensor(1.0000), 'topic': 2, 'score': 0.0}, {'checklist_item': 'ความแรง400มิลลิกรัม', 'matched_sentence': 'ความแรง400มิลลิกรัม', 'similarity_score': tensor(1.0000), 'topic': 3, 'score': 0.0}, {'checklist_item': 'จำนวน10เม็ด', 'matched_sentence': 'จำนวน10เม็ด', 'similarity_score': tensor(1.), 'topic': 4, 'score': 0.0}, {'checklist_item': 'ใช้เพื่อบรรเทาอาการปวด', 'matched_sentence': 'ใช้เพื่อบรรเทาอาการปวด', 'similarity_score': tensor(1.0000), 'topic': 5, 'score': 0.0}, {'checklist_item': 'รับประทานครั้งละ1เม็ด', 'matched_sentence': 'รับประทานครั้งละ1เม็ด', 'similarity_score': tensor(1.), 'topic': 6, 'score': 0.0}, {'checklist_item': 'วันละ3ครั้ง', 'matched_sentence': 'วันละ3ครั้ง', 'similarity_score': tensor(1.0000), 'topic': 7, 'score': 0.0}, {'checklist_item': 'หลังอาหารเช้ากลางวันเย็น', 'matched_sentence': 'หลังอาหารเช้ากลางวันเย็น', 'similarity_score': tensor(1.), 'topic': 8, 'score': 0.0}, {'che

test model

In [9]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
recording = []

# Encode the checklist sentences
checklit_embeddings = model.encode(checklist)

# Iterate through split_sent and check for similarity
for sent in split_sent:
    sent_embedding = model.encode(sent)
    similarities = util.cos_sim(sent_embedding, checklit_embeddings)

    # อัปเดตผลลัพธ์หากเจอข้อความที่ตรงกับขั้นตอนมาตรฐาน
    for i, step in enumerate(checklist):
      
        recording.append({
            "checklist_item": checklist[i],
            "matched_sentence": sent,
            "similarity_score": similarities[0][i],
            "topic": topic[i],
            "score": score[i],
        })

# Save results to an Excel file
df = pd.DataFrame(recording)
df.to_excel("recording_results.xlsx", index=False)

# Print the recorded matches
print(recording)

[{'checklist_item': 'ผมเป็นเภสัชกร', 'matched_sentence': 'วันนี้มารับยา', 'similarity_score': tensor(0.5094), 'topic': 0, 'score': 1.0}, {'checklist_item': 'ผู้ป่วยชื่ออะไร', 'matched_sentence': 'วันนี้มารับยา', 'similarity_score': tensor(0.3116), 'topic': 1, 'score': 1.0}, {'checklist_item': 'ผู้ป่วยชื่อชาตินามสกุลอะไร', 'matched_sentence': 'วันนี้มารับยา', 'similarity_score': tensor(0.2984), 'topic': 1, 'score': 0.5}, {'checklist_item': 'ผู้ป่วยชื่อชาติตระการตาใช่ไหม', 'matched_sentence': 'วันนี้มารับยา', 'similarity_score': tensor(0.4396), 'topic': 1, 'score': 0.25}, {'checklist_item': 'ผู้ป่วยชื่อนามสกุลอะไร', 'matched_sentence': 'วันนี้มารับยา', 'similarity_score': tensor(0.2865), 'topic': 1, 'score': 1.0}, {'checklist_item': 'ไอบูโปรเพน', 'matched_sentence': 'วันนี้มารับยา', 'similarity_score': tensor(0.3425), 'topic': 2, 'score': 0.0}, {'checklist_item': 'ความแรง400มิลลิกรัม', 'matched_sentence': 'วันนี้มารับยา', 'similarity_score': tensor(0.0390), 'topic': 3, 'score': 0.0}, {'c

# try to using fine tuning model

In [10]:
import psycopg2
from psycopg2.extras import RealDictCursor

def get_db_connection():
    conn = psycopg2.connect(
        dbname='Testing',
        user='postgres',
        password='123456789',
        host='localhost'
    )
    return conn

In [11]:
def get_data():
    conn = get_db_connection()
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute('SELECT * FROM processing order by topic')
    processing = cursor.fetchall()
    cursor.close()
    conn.close()
    return processing

In [12]:
data = get_data()
checklist = [item['checklist'] for item in data]
topic = [item['topic'] for item in data]
score = [item['score'] for item in data]
request = [item['prerequest'] for item in data]

In [13]:
from pythainlp.tokenize import word_tokenize
from pythainlp.util import Trie

In [14]:
text = [
    "สวัสดีค่ะหนูเป็นเภสัชกรชื่อสมหญิงใจดีคนไข้ชื่อนามสกุลอะไรคะ",
    "คนไข้มาด้วยอาการปวดหูใช่ไหมคะ",
    """วันนี้มารับยา3ตัวนะค่ะยานี้ชื่อไอบูโปรเพนความแรง400มิลลิกรัมจำนวน10เม็ดใช้เพื่อบรรเทาอาการปวดรับประทานครั้งละ1เม็ดวันละ3ครั้งหลังอาหารเช้ากลางวันเย็นยานี้ควรรับประทานหลังอาหารทันทีส่วนยานี้ชื่ออะมอกซีซิลินความแรง500มิลลิกรัมจำนวน30เม็ดใช้รักษาอาการติดเชื้อที่หูรับประทานครั้งละ1เม็ดวันละ3ครั้งหลังอาหารเช้ากลางวันเย็นยานี้ควรรับประทานติดต่อกันทุกวันจนหมดและยานี้ชื่อเด็กซ์ออฟจำนวน1ขวดใช้สำหรับหยอดหูเพื่อรักษาอาการติดเชื้อที่หูหยอด3หยดที่หูข้างซ้ายวันละ3ครั้งหลังอาหารเช้ากลางวันเย็นซึ่งยานี้มีวิธีการใช้ในลักษณะพิเศษเดี๋ยวฉันสอนวิธีการใช้งานให้นะคะเริ่มด้วยการล้างมือให้สะอาดก่อนใช้ยาจากนั้นใช้ไม้พันสำลีเช็ดหนองในใบหูหรือรูหูให้สะอาดก่อนและหลังหยอดยาต่อมาให้หยิบยาจากกล่องมาถือไว้ที่มือและเขย่าขวดก่อนใช้ทุกครั้งถัดไปให้คนไข้นอนตะแคงหรือนั่งเอียงศีรษะให้หูข้างที่จะหยอดยาอยู่ด้านบนถัดมาให้ดึงใบหูข้างที่จะหยอดยาไปด้านหลังและดึงขึ้นด้านบนโดยหยอดยาเข้าไปในหูตามที่แพทย์สั่งระวังอย่าเอาหลอดหยดสอดเข้าไปในรูหูให้คนไข้เอียงตะแคงอยู่ท่าเดิมประมาณ3ถึง5นาทีซึ่งอาจจะเอาสำลีใส่ในรูหูไว้อีกระยะหนึ่งเพื่อไม่ให้ยาไหลออกมาหากเก็บยาหยอดหูไว้ในตู้เย็นก่อนใช้ยาให้กำหยอดยาไว้ในอุ้งมือสักครู่เพื่อปรับอุณหภูมิให้ใกล้เคียงกับร่างกายก่อนหยอด""",
]

In [15]:
# Define custom words
custom_words = [
    "คนไข้ชื่อนามสกุลอะไร","สวัสดี","ไอบูโปรเพน","ความแรง400มิลลิกรัม","จำนวน10เม็ด","รับประทานครั้งละ1เม็ด","วันละ3ครั้ง","หลังอาหารเช้ากลางวันเย็น",
    "ควรรับประทานหลังอาหารทันที","อะมอกซีซิลิน","ความแรง500มิลลิกรัม","จำนวน30เม็ด","ควรรับประทานติดต่อกันทุกวันจนหมด","เด็กซ์ออฟ","จำนวน1ขวด",
    "หยอด3หยดที่หูข้างซ้าย","ล้างมือให้สะอาด","นอนตะแคงหรือนั่งเอียงศีรษะให้หูข้างที่จะหยอดยาอยู่ด้านบน","ให้ดึงใบหูข้างที่จะหยอดยาไปด้านหลังและดึงขึ้นด้านบน",
    "หยอดยาเข้าไปในหูตามที่แพทย์สั่ง","ระวังอย่าเอาหลอดหยดสอดเข้าไปในรูหู","เอียงตะแคงอยู่ท่าเดิมประมาณ3ถึง5นาที","สำลีใส่ในรูหูไว้อีกระยะหนึ่งเพื่อไม่ให้ยาไหลออกมา",
    "ให้เขย่าขวดก่อนใช้ทุกครั้ง","ใช้ไม้พันสำลีเช็ดหนองในใบหูหรือรูหูให้สะอาดก่อนและหลังหยอดยา","หากเก็บไว้ในตู้เย็นก่อนใช้ยาให้กำหลอดยาไว้ในอุ้งมือสักครู่",
    "เพื่อปรับอุณหภูมิให้ใกล้เคียงกับร่างกายก่อนหยอด","เขย่าขวดก่อนใช้ทุกครั้ง",
    ]
# Create a Trie for custom dictionary
trie = Trie(custom_words)

In [16]:
split_sent = word_tokenize(text[2], custom_dict=trie)
print(split_sent)

['วันนี้มารับยา', '3', 'ตัวนะค่ะยานี้ชื่อ', 'ไอบูโปรเพน', 'ความแรง400มิลลิกรัม', 'จำนวน10เม็ด', 'ใช้เพื่อบรรเทาอาการปวด', 'รับประทานครั้งละ1เม็ด', 'วันละ3ครั้ง', 'หลังอาหารเช้ากลางวันเย็น', 'ยานี้', 'ควรรับประทานหลังอาหารทันที', 'ส่วนยานี้ชื่อ', 'อะมอกซีซิลิน', 'ความแรง500มิลลิกรัม', 'จำนวน30เม็ด', 'ใช้รักษาอาการติดเชื้อที่หู', 'รับประทานครั้งละ1เม็ด', 'วันละ3ครั้ง', 'หลังอาหารเช้ากลางวันเย็น', 'ยานี้', 'ควรรับประทานติดต่อกันทุกวันจนหมด', 'และยานี้ชื่อ', 'เด็กซ์ออฟ', 'จำนวน1ขวด', 'ใช้สำหรับหยอดหูเพื่อรักษาอาการติดเชื้อที่หู', 'หยอด3หยดที่หูข้างซ้าย', 'วันละ3ครั้ง', 'หลังอาหารเช้ากลางวันเย็น', 'ซึ่งยานี้มีวิธีการใช้ในลักษณะพิเศษเดี๋ยวฉันสอนวิธีการใช้งานให้นะคะเริ่มด้วยการ', 'ล้างมือให้สะอาด', 'ก่อนใช้ยาจากนั้น', 'ใช้ไม้พันสำลีเช็ดหนองในใบหูหรือรูหูให้สะอาดก่อนและหลังหยอดยา', 'ต่อมาให้หยิบยาจากกล่องมาถือไว้ที่มือและ', 'เขย่าขวดก่อนใช้ทุกครั้ง', 'ถัดไปให้คนไข้', 'นอนตะแคงหรือนั่งเอียงศีรษะให้หูข้างที่จะหยอดยาอยู่ด้านบน', 'ถัดมา', 'ให้ดึงใบหูข้างที่จะหยอดยาไปด้านหลังและดึงขึ้นด้านบน', 'โดย

In [20]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

model = SentenceTransformer("trained_model")
threshold = 0.75
recording = []
count = 0
matched_topics = {-1} # Track matched topics to avoid duplicate scoring

# Track whether each checklist item is matched
checklist_status = [False] * len(checklist)

# Encode the checklist sentences
checklit_embeddings = model.encode(checklist)

# Iterate through split_sent and check for similarity
for sent in split_sent:
    sent_embedding = model.encode(sent)
    similarities = util.cos_sim(sent_embedding, checklit_embeddings)

    # best_match_index = similarities[0].argmax().item()  # Get index of highest similarity
    # best_score = similarities[0][best_match_index].item()

    for i, check in enumerate(checklist):
        if similarities[0][i] > threshold:
            best_match_index = i
            matched_checklist_item = checklist[best_match_index]
            matched_topic = topic[best_match_index]
            matched_score = score[best_match_index]
            required_topic = request[best_match_index]

            # Ensure dependencies are met
            if required_topic != -1 and required_topic not in matched_topics:
                continue  # Skip if required topic was not mentioned first

            checklist_status[best_match_index] = True  # Mark as matched

            # Add score only if the topic has not been counted before
            if matched_topic not in matched_topics:
                count += matched_score
                matched_topics.add(matched_topic)

            recording.append({
                "checklist_item": matched_checklist_item,
                "matched_sentence": sent,
                "similarity_score": similarities[0][i],
                "topic": matched_topic,
                "score": matched_score
            })

# Save results to an Excel file
df = pd.DataFrame(recording)
df.to_excel("recording_results.xlsx", index=False)

# Print the recorded matches
print(recording)
print("Checklist Status:")
for i,check in enumerate(checklist):
    print(check,":",checklist_status[i])
print("Total Score:", count)

[{'checklist_item': 'ไอบูโปรเพน', 'matched_sentence': 'ไอบูโปรเพน', 'similarity_score': tensor(1.0000), 'topic': 2, 'score': 0.0}, {'checklist_item': 'ความแรง400มิลลิกรัม', 'matched_sentence': 'ความแรง400มิลลิกรัม', 'similarity_score': tensor(1.0000), 'topic': 3, 'score': 0.0}, {'checklist_item': 'จำนวน10เม็ด', 'matched_sentence': 'จำนวน10เม็ด', 'similarity_score': tensor(1.0000), 'topic': 4, 'score': 0.0}, {'checklist_item': 'ใช้เพื่อบรรเทาอาการปวด', 'matched_sentence': 'ใช้เพื่อบรรเทาอาการปวด', 'similarity_score': tensor(1.), 'topic': 5, 'score': 0.0}, {'checklist_item': 'รับประทานครั้งละ1เม็ด', 'matched_sentence': 'รับประทานครั้งละ1เม็ด', 'similarity_score': tensor(1.), 'topic': 6, 'score': 0.0}, {'checklist_item': 'วันละ3ครั้ง', 'matched_sentence': 'วันละ3ครั้ง', 'similarity_score': tensor(1.), 'topic': 7, 'score': 0.0}, {'checklist_item': 'หลังอาหารเช้ากลางวันเย็น', 'matched_sentence': 'หลังอาหารเช้ากลางวันเย็น', 'similarity_score': tensor(1.0000), 'topic': 8, 'score': 0.0}, {'che

In [60]:
matched_topics

{-1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33}

# Nomic v2

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)
sentences = ['ผู้ป่วยชื่ออะไร', "ให้คนไข้"]
embeddings = model.encode(sentences, prompt_name="passage")
print(embeddings.shape)
# (2, 768)

similarity = model.similarity(embeddings[0], embeddings[1])
print(similarity)
# tensor([[0.9118]])

(2, 768)
tensor([[0.8261]])


In [5]:
model.eval()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: NomicBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [6]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model.eval()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

# save data to database

In [1]:
import psycopg2
from psycopg2.extras import RealDictCursor

def get_db_connection():
    conn = psycopg2.connect(
        dbname='ScoringT',
        user='postgres',
        password='123456789',
        host='localhost'
    )
    return conn

conn = get_db_connection()

cur = conn.cursor()

# Insert multiple rows with ID_checklist from 1 to 27
id_situ = 24
#6 แพ้
#40 โรคประจำตัว
arr = [1, 2, 3, 4, 5, 6,175,426,177,208,179,180,181,*range(401,425)]

for id_checklist in range(0, len(arr)):  # 1 to 27 inclusive
    cur.execute(
        'INSERT INTO "having" ("ID_having", "ID_situ", "ID_checklist") VALUES (%s, %s, %s)',
        (id_checklist+738, id_situ, arr[id_checklist])
    )

# Commit and close the connection
conn.commit()
cur.close()
conn.close()

UniqueViolation: duplicate key value violates unique constraint "having_pkey"
DETAIL:  Key ("ID_having")=(738) already exists.


In [2]:
import psycopg2
from psycopg2.extras import RealDictCursor

def get_db_connection():
    conn = psycopg2.connect(
        dbname='ScoringT',
        user='postgres',
        password='123456789',
        host='localhost'
    )
    return conn

conn = get_db_connection()

cur = conn.cursor()

# Insert multiple rows with ID_checklist from 1 to 27
id_situ = 24

for id_checklist in range(0,len(custom_words)):  # 1 to 27 inclusive
    cur.execute(
        'INSERT INTO public.semantic_token_situ("ID_token", sem_word_token, "ID_situ") VALUES(%s, %s, %s);',(id_checklist+588, custom_words[id_checklist], id_situ)
    )

# Commit and close the connection
conn.commit()
cur.close()
conn.close()

NameError: name 'custom_words' is not defined

In [3]:
import psycopg2
from psycopg2.extras import RealDictCursor

def get_db_connection():
    conn = psycopg2.connect(
        dbname='ScoringT',
        user='postgres',
        password='123456789',
        host='localhost'
    )
    return conn

conn = get_db_connection()

cur = conn.cursor()

# Insert multiple rows with ID_checklist from 1 to 27
id_situ = 24

for id_checklist in range(0,len(new_words)):  # 1 to 27 inclusive
    cur.execute(
        'INSERT INTO public.token_situ("ID_token", word_token, "ID_situ") VALUES (%s, %s, %s);',(id_checklist+715, new_words[id_checklist], id_situ)
    )

# Commit and close the connection
conn.commit()
cur.close()
conn.close()

NameError: name 'new_words' is not defined

In [23]:
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
from itertools import combinations

# Connect to PostgreSQL
def get_db_connection():
    return psycopg2.connect(
        dbname='ScoringT',
        user='postgres',
        password='123456789',
        host='localhost'
    )

# Step 1: ดึงข้อมูลจาก checklist ที่ topic = 4
conn = get_db_connection()
cur = conn.cursor(cursor_factory=RealDictCursor)
cur.execute('SELECT * FROM public.checklist WHERE topic = 4 ORDER BY "ID_checklist" ASC')
rows = cur.fetchall()

# Step 2: สร้างคู่ที่ไม่ซ้ำกันแบบ (A,B) เท่านั้น (ไม่มี (B,A))
pairs = list(combinations(rows, 2))  # combinations จาก itertools จับแบบไม่ซ้ำ

# Step 3: เตรียมข้อมูลสำหรับ Excel
pair_data = []
for a, b in pairs:
    pair_data.append({
        "ID_checklist_1": a["ID_checklist"],
        "Detail_1": a["check_sent"],
        "ID_checklist_2": b["ID_checklist"],
        "Detail_2": b["check_sent"]
    })

# Step 4: สร้าง DataFrame และบันทึกลง Excel
df = pd.DataFrame(pair_data)
df.to_excel("checklist_pairs.xlsx", index=False)

# ปิดการเชื่อมต่อ
cur.close()
conn.close()

# new word for save into database all situ

In [120]:
import psycopg2
from psycopg2.extras import RealDictCursor

def get_db_connection():
    conn = psycopg2.connect(
        dbname='ScoringT',
        user='postgres',
        password='123456789',
        host='localhost'
    )
    return conn

conn = get_db_connection()
cur = conn.cursor()

new_word = 'มีความแรงของยา'

for ID_situ in range(1, 25):
    cur.execute(
        '''
        INSERT INTO public.token_situ(word_token, "ID_situ")
        VALUES (%s, %s);
        ''',
        (new_word, ID_situ)
    )

conn.commit()
cur.close()
conn.close()

In [121]:
import psycopg2
from psycopg2.extras import RealDictCursor

def get_db_connection():
    conn = psycopg2.connect(
        dbname='ScoringT',
        user='postgres',
        password='123456789',
        host='localhost'
    )
    return conn

conn = get_db_connection()
cur = conn.cursor()

new_word = 'มีความแรงของยา'

for ID_situ in range(1, 25):
    cur.execute(
        '''
        INSERT INTO public.semantic_token_situ(sem_word_token, "ID_situ")
        VALUES (%s, %s);
        ''',
        (new_word, ID_situ)
    )

conn.commit()
cur.close()
conn.close()

In [6]:
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd

def get_db_connection():
    conn = psycopg2.connect(
        dbname='ScoringT',
        user='postgres',
        password='123456789',
        host='localhost'
    )
    return conn

# อ่านข้อมูลจากไฟล์ Excel
df = pd.read_excel('./data/new_vocab.xlsx')

# เชื่อมต่อกับฐานข้อมูล
conn = get_db_connection()
cur = conn.cursor()

# วนลูปแต่ละคำจาก Excel และสถานการณ์ ID_situ
for new_word in df['Vocab']:
    for ID_situ in range(1, 25):
        # ตรวจสอบว่าคำนี้มีอยู่แล้วกับ ID_situ นี้หรือยัง
        cur.execute(
            '''
            SELECT 1 FROM public.token_situ
            WHERE word_token = %s AND "ID_situ" = %s;
            ''',
            (new_word, ID_situ)
        )
        exists = cur.fetchone()

        # ถ้าไม่พบ (None) จึงค่อยเพิ่มข้อมูล
        if not exists:
            cur.execute(
                '''
                INSERT INTO public.token_situ(word_token, "ID_situ")
                VALUES (%s, %s);
                ''',
                (new_word, ID_situ)
            )

# บันทึกการเปลี่ยนแปลง
conn.commit()

# ปิด cursor และการเชื่อมต่อ
cur.close()
conn.close()

# problem of model

In [1]:
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
from itertools import combinations

# Connect to PostgreSQL
def get_db_connection():
    return psycopg2.connect(
        dbname='ScoringT',
        user='postgres',
        password='123456789',
        host='localhost'
    )

# Step 1: ดึงข้อมูลจาก checklist ที่ topic = 4
conn = get_db_connection()
cur = conn.cursor(cursor_factory=RealDictCursor)
cur.execute('SELECT * FROM public.checklist WHERE topic = 4 ORDER BY "ID_checklist" ASC')
rows = cur.fetchall()

# Step 2: สร้างคู่ที่ไม่ซ้ำกันแบบ (A,B) เท่านั้น (ไม่มี (B,A))
pairs = list(combinations(rows, 2))  # combinations จาก itertools จับแบบไม่ซ้ำ

# Step 3: เตรียมข้อมูลสำหรับ Excel
pair_data = []
for a, b in pairs:
    pair_data.append({
        "ID_checklist_1": a["ID_checklist"],
        "Detail_1": a["check_sent"],
        "ID_checklist_2": b["ID_checklist"],
        "Detail_2": b["check_sent"]
    })

# ปิดการเชื่อมต่อ
cur.close()
conn.close()

In [3]:
from sentence_transformers import SentenceTransformer, util

# โหลดโมเดล
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# สร้าง similarity สำหรับทุกคู่
for pair in pair_data:
    sent1 = pair['Detail_1']
    sent2 = pair['Detail_2']
    
    # สร้าง embedding
    embeddings = model.encode([sent1, sent2], convert_to_tensor=True)

    # คำนวณ similarity
    similarity = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()
    
    # เพิ่มลงใน dictionary
    pair['similarity'] = similarity

# แปลงเป็น DataFrame และส่งออก Excel
df = pd.DataFrame(pair_data)
df.to_excel("checklist_similarity.xlsx", index=False)

print("Saved similarity results to checklist_similarity.xlsx")

Saved similarity results to checklist_similarity.xlsx


# save image in situ

In [7]:
from PIL import Image
from glob import glob
import os
import psycopg2
from psycopg2.extras import RealDictCursor

def get_db_connection():
    conn = psycopg2.connect(
        dbname='ScoringT',
        user='postgres',
        password='123456789',
        host='localhost'
    )
    return conn

# Connect to the database
conn = get_db_connection()
cur = conn.cursor()

# Folder containing the images
path = r'C:\Users\LENOVO\Desktop\DemoP\VC\image_ใบสั่งยา'

# Get all PNG image file paths
image_files = glob(os.path.join(path, '*.png'))

# Loop through each image file and update the database
for img_path in image_files:
    with open(img_path, 'rb') as f:
        binary_data = f.read()

    # Extract the situation name from the filename (without extension)
    name_situ = os.path.splitext(os.path.basename(img_path))[0]

    # Print to confirm what’s being processed
    print(f"Updating image for: {name_situ}")

    # Update the 'image' field in the 'public.situation' table
    cur.execute(
        """
        UPDATE public.situation
        SET image = %s
        WHERE name_situ = %s
        """,
        (binary_data, name_situ)
    )

# Commit changes and close the connection
conn.commit()
cur.close()
conn.close()

print("Image updates completed successfully.")

Updating image for: การจ่ายยาคุมกำเนิด1
Updating image for: การจ่ายยาคุมกำเนิด2
Updating image for: การจ่ายยาฉีดอินซูลิน1
Updating image for: การจ่ายยาฉีดอินซูลิน2
Updating image for: การจ่ายยาน้ำเกลือล้างจมูก1
Updating image for: การจ่ายยาน้ำเกลือล้างจมูก2
Updating image for: การจ่ายยาผงผสมน้ำ1
Updating image for: การจ่ายยาผงผสมน้ำ2
Updating image for: การจ่ายยาพ่นจมูก1
Updating image for: การจ่ายยาพ่นจมูก2
Updating image for: การจ่ายยาพ่นรูปแบบDPI1(accuhaler)
Updating image for: การจ่ายยาพ่นรูปแบบDPI2(turbuhaler)
Updating image for: การจ่ายยาพ่นรูปแบบMDI1
Updating image for: การจ่ายยาพ่นรูปแบบMDI2
Updating image for: การจ่ายยาหยอดตา1
Updating image for: การจ่ายยาหยอดตา2
Updating image for: การจ่ายยาหยอดหู1
Updating image for: การจ่ายยาหยอดหู2
Updating image for: การจ่ายยาเหน็บช่องคลอด1
Updating image for: การจ่ายยาเหน็บช่องคลอด2
Updating image for: การจ่ายยาเหน็บทวาร1
Updating image for: การจ่ายยาเหน็บทวาร2
Updating image for: การจ่ายยาแผ่นแปะ1
Updating image for: การจ่ายยาแผ่นแปะ2
I